# Part B

## Step 1 - Adding Features

#### Load Data

In [ ]:
import pandas as pd

# !gdown 13HQpkd3ZExNBHiGTi4PVJJ1dn3SKV2MI
# hotels_file_path = "/content/hotels_data.csv"  
hotels_file_path = "./hotels_data.csv"  
df = pd.read_csv(hotels_file_path)

df.head()

#### Add columns and load to new CSV

In [ ]:
#Ensure dates are in correct format
df['Snapshot Date'] = pd.to_datetime(df['Snapshot Date'])
df['Checkin Date'] = pd.to_datetime(df['Checkin Date'])

df['DayDiff'] = (df['Checkin Date'] - df['Snapshot Date']).dt.days
df['WeekDay'] = df['Checkin Date'].dt.day_name()
df['DiscountDiff'] = df['Original Price'] - df['Discount Price']
df['DiscountPerc'] = (df['DiscountDiff'] / df['Original Price']) * 100

display(df.head())


**New csv format**

| Column Name       | Description                                                                                         | Example Value                       |
|--------------------|-----------------------------------------------------------------------------------------------------|-------------------------------------|
| **Snapshot ID**    | Unique identifier for each snapshot of data                                                        | 1                                   |
| **Snapshot Date**  | The date when the snapshot was taken                                                               | 2015-07-17                          |
| **Checkin Date**   | The date of check-in for the hotel                                                                 | 2015-08-12                          |
| **Days**           | Duration of the stay in days                                                                       | 5                                   |
| **Original Price** | Price of the stay without any discount (in dollars)                                                | 1178                                |
| **Discount Price** | Price of the stay after applying the discount (in dollars)                                         | 1040                                |
| **Discount Code**  | Code representing the type of discount applied (values 1-4, with 1 indicating no discount possible) | 1                                   |
| **Available Rooms**| Number of rooms available at the specified check-in date                                           | 6                                   |
| **Hotel Name**     | Name of the hotel                                                                                  | Best Western Plus Seaport Inn Downtown |
| **Hotel Stars**    | Star rating of the hotel                                                                           | 3                                   |
| **DayDiff**        | Number of days between the Snapshot Date and Checkin Date                                          | 26                                  |
| **WeekDay**        | Day of the week corresponding to the Checkin Date                                                  | Wednesday                           |
| **DiscountDiff**   | Difference between the Original Price and Discount Price (in dollars)                              | 138                                 |
| **DiscountPerc**   | Percentage of discount applied, calculated as `(DiscountDiff / Original Price) * 100`             | 11.714770797962649                           |


#### Save To CSV

In [ ]:
changed_hotels_path = "./hotels_data_changed.csv"
#df.to_csv(changed_hotels_path, index=False)

## (Shared) -  Feature Engineering

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

def convert_weekday_to_num(weekday):
    weekday_mapping = {
        'Monday': 0,
        'Tuesday': 1,
        'Wednesday': 2,
        'Thursday': 3,
        'Friday': 4,
        'Saturday': 5,
        'Sunday': 6
    }
    return weekday_mapping.get(weekday, np.nan)



def feature_engineering(df):
    """
    Process the raw hotel data DataFrame by converting dates, recalculating date differences,
    and mapping the weekday to numeric.
    
    This function adds:
      - 'Snapshot_Date_num': Numeric representation of 'Snapshot Date'
      - 'Checkin_Date_num' : Numeric representation of 'Checkin Date'
      - 'DayDiff'          : Recalculated difference in days between check-in and snapshot
      - 'WeekDay_num'      : Numeric representation of the 'WeekDay'
    
    Args:
        df (pd.DataFrame): Raw input DataFrame.
        
    Returns:
        pd.DataFrame: The DataFrame with additional engineered features.
    """

    # Convert date strings to datetime objects
    df['Snapshot Date'] = pd.to_datetime(df['Snapshot Date'])
    df['Checkin Date'] = pd.to_datetime(df['Checkin Date'])
    
    # Create new numeric date columns using ordinal representation
    df['Snapshot_Date_num'] = df['Snapshot Date'].apply(lambda x: x.toordinal())
    df['Checkin_Date_num'] = df['Checkin Date'].apply(lambda x: x.toordinal())
    
    df['DayDiff'] = (df['Checkin Date'] - df['Snapshot Date']).dt.days
    
    df['WeekDay_num'] = df['WeekDay'].apply(convert_weekday_to_num)
    
    return df



def encode_categorical_columns(df, categorical_cols):
    """
    One-hot encode specified categorical columns.
    
    Args:
        df (pd.DataFrame): DataFrame containing the data.
        categorical_cols (list of str): List of columns to one-hot encode.
        
    Returns:
        pd.DataFrame: DataFrame with categorical columns encoded.
    """
    df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
    return df_encoded



def normalize_features(df, features_to_scale, scaler=None):
    """
    Normalize specified features in the DataFrame using a given scaler.
    
    Args:
        df (pd.DataFrame): The input data frame.
        features_to_scale (list): List of column names to normalize.
        scaler (sklearn.preprocessing object, optional): A scaler instance. 
            If None, StandardScaler is used.
    
    Returns:
        df_norm (pd.DataFrame): DataFrame with normalized features.
        scaler (sklearn.preprocessing object): The fitted scaler.
    """
    if scaler is None:
        scaler = StandardScaler()
    
    df_norm = df.copy()
    
    df_norm[features_to_scale] = scaler.fit_transform(df_norm[features_to_scale])
    
    return df_norm, scaler

## Step 2 - Best Discount Code

### 2.A) Data Preperation

In [ ]:
# Drop irrelevant columns

df = pd.read_csv(changed_hotels_path)
original_df = df
df = df[['WeekDay', 'Snapshot Date', 'Checkin Date', 'DayDiff', 'Hotel Name', 'Discount Code']] 
df = df.rename(columns={'Discount Code': 'Class'})
df.head()

In [ ]:
# Feature Engineering

def map_days_to_numbers(df: pd.DataFrame) -> pd.DataFrame:
    weekday_mapping = {'Sunday': 1, 'Monday': 2, 'Tuesday': 3, 'Wednesday': 4, 'Thursday': 5, 'Friday': 6, 'Saturday': 7}
    df['WeekDay'] = df['WeekDay'].map(weekday_mapping)
    return df

def map_hotel_names_to_numbers(df: pd.DataFrame) -> pd.DataFrame:
    hotel_mapping = {hotel: i for i, hotel in enumerate(df['Hotel Name'].unique())}
    df['Hotel_Index'] = df['Hotel Name'].map(hotel_mapping)
    df.drop(['Hotel Name'], axis=1, inplace=True)
    return df

def map_date_to_numbers(df: pd.DataFrame, old_coloumn: str, new_coloumn: str) -> pd.DataFrame:
    df[old_coloumn] = pd.to_datetime(df[old_coloumn])
    df[f'{new_coloumn}_Year'] = df[old_coloumn].dt.year
    df[f'{new_coloumn}_Month'] = df[old_coloumn].dt.month
    df[f'{new_coloumn}_Day'] = df[old_coloumn].dt.day
    df.drop([old_coloumn], axis=1, inplace=True)

    return df

In [ ]:
df = map_days_to_numbers(df)
df = map_hotel_names_to_numbers(df)
df = map_date_to_numbers(df, 'Snapshot Date', 'Snapshot')
df = map_date_to_numbers(df, 'Checkin Date', 'Checkin')
df.head()

In [ ]:
# Plot Distributions

import matplotlib.pyplot as plt
import seaborn as sns

def plot_features_distribution(df: pd.DataFrame) -> None:
    fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(12, 15))
    fig.suptitle('Distribution of Features')

    for i, column in enumerate(df.columns):
        row = i // 3  # Calculate row index
        col = i % 3    # Calculate column index
    
        sns.histplot(df[column], bins=10, ax=axes[row, col])
        axes[row, col].set_title(f'Distribution of {column}')
        axes[row, col].set_xlabel(column)
        axes[row, col].set_ylabel('Frequency')

    plt.tight_layout()
    plt.show()

plot_features_distribution(df)

In [ ]:
# Drop Irrelevant Features
# We see that 2016 doesnt have many values and it can manipulte the data.

df = df[df['Checkin_Year'] != 2016]
df = df.drop(columns=['Checkin_Year'])

df = df[df['Snapshot_Year'] != 2016]
df = df.drop(columns=['Snapshot_Year'])

In [ ]:
plot_features_distribution(df)

In [ ]:
# check class imbalance

class_counts = df['Class'].value_counts()
print(class_counts)

In [ ]:
df.head()

### Utils

##### Imports

In [ ]:
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

from xgboost import XGBClassifier
import numpy as np
import pandas as pd
import json
from itertools import cycle
from imblearn.over_sampling import RandomOverSampler, ADASYN, SMOTE
from imblearn.under_sampling import RandomUnderSampler

##### Train Test Split

In [ ]:
def change_categrical_feilds(df):
  df['WeekDay'] = df['WeekDay'].astype('category')
  df['Hotel_Index'] = df['Hotel_Index'].astype('category')
  df['Snapshot_Month'] = df['Snapshot_Month'].astype('category')
  df['Checkin_Month'] = df['Checkin_Month'].astype('category')
  df['DayDiff'] = df['DayDiff'].astype('category')
  df['Snapshot_Day'] = df['Snapshot_Day'].astype('category')
  df['Checkin_Day'] = df['Checkin_Day'].astype('category')

  return df


def get_x_y_from_csv():
  hotels_predict_discount_path = "./hotels_predict_discount.csv"
  #hotels_predict_discount_path = "/content/hotels_predict_discount.csv"

  df = pd.read_csv(hotels_predict_discount_path)
  df = change_categrical_feilds(df)

  X = df[['WeekDay', 'DayDiff', 'Hotel_Index', 'Snapshot_Month', 'Snapshot_Day', 'Checkin_Month', 'Checkin_Day']]
  y = df['Class']

  return X, y


def get_train_test():
  X, y = get_x_y_from_csv()
  # XGBost expects [0,1,2,3] instead of [1,2,3,4]
  y = y - 1

  return train_test_split(X, y, test_size=0.2, random_state=42)


def get_smote_train_test():
  X, y = get_x_y_from_csv()
  smote = SMOTE(random_state=42, sampling_strategy='auto')
  X_resampled, y_resampled = smote.fit_resample(X, y)

  return train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


def get_undersample_train_test():
  X, y = get_x_y_from_csv()
  rus = RandomUnderSampler(random_state=42)
  X_resampled, y_resampled = rus.fit_resample(X, y)

  return train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

##### Evaluation

In [ ]:
from sklearn.metrics import accuracy_score

def evaluate_model(model, X_train, X_test, y_train, y_test):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    print_accuracy(y_train, y_train_pred, y_test, y_test_pred)


def print_accuracy(y_train, y_train_pred, y_test, y_test_pred):
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    print(f"  Train Accuracy: {train_accuracy:.2f}")
    print(f"  Test Accuracy: {test_accuracy:.2f}")

In [ ]:
def model_statistics(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (Macro): {precision:.4f}")
    print(f"Recall (Macro): {recall:.4f}")
    print(f"F1 Score (Macro): {f1:.4f}")

    cm = confusion_matrix(y_test, y_pred)
    cm_df = pd.DataFrame(cm, index=[f'Actual {i+1}' for i in range(cm.shape[0])],
                     columns=[f'Predicted {i+1}' for i in range(cm.shape[1])])
    print(cm_df)

    n_classes = cm.shape[0]

    TP = np.diag(cm)  # True Positives per class
    FP = cm.sum(axis=0) - np.diag(cm)  # False Positives per class
    FN = cm.sum(axis=1) - np.diag(cm)  # False Negatives per class

    TN = np.zeros(n_classes)
    for i in range(n_classes):
        # TN for class i: sum of all elements excluding row i and column i
        TN[i] = cm.sum() - (cm[i, :].sum() + cm[:, i].sum()) + cm[i, i]

    # Calculate TPR (True Positive Rate) and FPR (False Positive Rate) per class
    TPR = TP / (TP + FN)  # Sensitivity or Recall
    FPR = FP / (FP + TN)

    # Handle division by zero
    TPR = np.where((TP + FN) == 0, 0, TPR)
    FPR = np.where((FP + TN) == 0, 0, FPR)

    # Print results
    print("\nMetrics per Class:")
    for i in range(n_classes):
        print(f"Class {i+1}: True Positive Rate (TPR): {TPR[i]:.4f}, False Positive Rate (FPR): {FPR[i]:.4f}")

In [ ]:
def print_class_counts(model, X_test, y_test, y_train):
    y_pred = model.predict(X_test)

    y_train_counts = pd.Series(y_train).value_counts()
    y_test_counts = pd.Series(y_test).value_counts()
    unique_classes, y_pred_counts = np.unique(y_pred, return_counts=True)
    y_pred_counts = pd.Series(y_pred_counts, index=unique_classes)

    all_classes = sorted(list(set(y_train.unique()).union(set(y_test.unique())).union(set(y_pred))))

    data = {
        'Class': all_classes,
        'y_train_counts': [y_train_counts.get(c, 0) for c in all_classes],
        'y_test_counts': [y_test_counts.get(c, 0) for c in all_classes],
        'y_pred_counts': [y_pred_counts.get(c, 0) for c in all_classes]
    }

    df_counts = pd.DataFrame(data)
    df_counts['Class'] = df_counts['Class'] + 1

    display(df_counts)

In [ ]:
def evaluate__model_extended(model, X_test, y_test, model_name):
    n_classes = 4
    y_prob = model.predict_proba(X_test)
    
    # Binarize the output for ROC
    y_test_bin = label_binarize(y_test, classes=range(n_classes))
    
    # Initialize dictionaries for fpr, tpr, and roc_auc
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    # Compute ROC curve and ROC area for each class
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_prob.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    # Predict labels for other metrics
    y_pred = model.predict(X_test)
    
    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Calculate per-class FPR and FNR
    fpr_per_class = []
    fnr_per_class = []
    for i in range(n_classes):
        tn = np.sum(cm[:, :i].sum() + cm[:, i+1:].sum())  # True Negatives for class i
        fp = np.sum(cm[:, i]) - cm[i, i]  # False Positives for class i
        fn = cm[i, :].sum() - cm[i, i]  # False Negatives for class i
        tp = cm[i, i]  # True Positives for class i
        fpr_per_class.append(fp / (fp + tn) if (fp + tn) > 0 else 0)
        fnr_per_class.append(fn / (fn + tp) if (fn + tp) > 0 else 0)
    
    # Compute global metrics (macro average)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Print results
    print(f"\nMetrics for {model_name}:")
    print(f"ROC AUC (Micro): {roc_auc['micro']:.4f}")
    print(f"Per-class ROC AUC: {[f'{i}: {roc_auc[i]:.4f}' for i in range(n_classes)]}")
    print(f"Per-class FPR: {[f'{i}: {fpr_per_class[i]:.4f}' for i in range(n_classes)]}")
    print(f"Per-class FNR: {[f'{i}: {fnr_per_class[i]:.4f}' for i in range(n_classes)]}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (Macro): {precision:.4f}")
    print(f"Recall (Macro): {recall:.4f}")
    print(f"F1 Score (Macro): {f1:.4f}")
    
    return fpr, tpr, roc_auc, y_pred, cm

##### Printing Params

In [ ]:
def print_naive_bayes_params(nb):
    params = nb.get_params()

    print(f"  Var smoothing: {params['var_smoothing']}")

In [ ]:
def print_decision_tree_params(dt):
    params = dt.get_params()

    print(f"  Criterion: {params['criterion']}")
    print(f"  Max Depth: {params['max_depth']}")
    print(f"  Depth: {dt.get_depth()}")
    print(f"  Num Leaves: {dt.get_n_leaves()}")
    print(f"  Min Samples Split: {params['min_samples_split']}")
    print(f"  Min Samples Leaf: {params['min_samples_leaf']}")
    print(f"  Max Features: {params['max_features']}")
    print(f"  Max Leaf Nodes: {params['max_leaf_nodes']}")
    print(f"  Min Impurity Decrease: {params['min_impurity_decrease']}")
    print(f"  CCP Alpha: {params['ccp_alpha']}")


In [ ]:
def print_random_forest_params(rf_classifier):
    params = rf_classifier.get_params()

    print(f"  N Estimators: {params['n_estimators']}")
    print(f"  Number of Leaves: {sum(tree.get_n_leaves() for tree in rf_classifier.estimators_)}")
    print(f"  Criterion: {params['criterion']}")
    print(f"  Max Depth: {params['max_depth']}")
    print(f"  Max Features: {params['max_features']}")
    print(f"  Max Leaf Nodes: {params['max_leaf_nodes']}")
    print(f"  Max Samples: {params['max_samples']}")
    print(f"  Min Impurity Decrease: {params['min_impurity_decrease']}")
    print(f"  Min Samples Leaf: {params['min_samples_leaf']}")
    print(f"  Min Samples Split: {params['min_samples_split']}")
    print(f"  Min Weight Fraction Leaf: {params['min_weight_fraction_leaf']}")
    print(f"  Parameters: {params}")


In [ ]:
def print_xgb_params(xgb):
  booster = xgb.get_booster()

  config_str = booster.save_config()
  config_dict = json.loads(config_str)

  learner_params = config_dict['learner']

  print(f" max_depth: {learner_params['gradient_booster']['tree_train_param']['max_depth']}")
  print(f" learning_rate: {learner_params['gradient_booster']['tree_train_param']['learning_rate']:.3}")

### 2.B) Algorithms

Implement and evaluate the following five algorithms: Random Forest, Decision Tree, Naïve Bayes, XGBoost, and a simple Random classifier.
For each algorithm, you need to experiment with different parameter settings to find the optimal combination that yields the best performance. 
Explanation of how each chosen parameter affects the algorithm's performance. 

#### Naive Bayes

##### Default Params

In [ ]:
X_train, X_test, y_train, y_test = get_train_test()
nb_categorial = CategoricalNB()
nb_categorial.fit(X_train, y_train)

evaluate_model(nb_categorial, X_train, X_test, y_train, y_test)

In [ ]:
print_class_counts(nb_categorial, X_test, y_test, y_train)

In [ ]:
model_statistics(nb_categorial, X_test, y_test)

##### Hyperparaeters tuning

Last model (with default prior=true) gave class 4 little percentage

In [ ]:
X_train, X_test, y_train, y_test = get_smote_train_test()
nb_categorial_smote = CategoricalNB(fit_prior=False)
nb_categorial.fit(X_train, y_train)

model_statistics(nb_categorial, X_test, y_test)

no class imbalance but less accurate

class sklearn.naive_bayes.CategoricalNB(*, alpha=1.0, force_alpha=True, fit_prior=True, class_prior=None, min_categories=None)

grid_search for alpha(var_smoothing) wasnt better

#### Decision Tree

##### Default Params

In [ ]:
X_train, X_test, y_train, y_test = get_train_test()

dt_classifier = DecisionTreeClassifier(random_state=42) 
dt_classifier.fit(X_train, y_train)

evaluate_model(dt_classifier, X_train, X_test, y_train, y_test)
print_decision_tree_params(dt_classifier)

##### HyperParameter Tuning


DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, ccp_alpha=0.0, monotonic_cst=None)[source]

**criterion: ['gini', 'entropy']** Gini and Entropy are two different ways to calculate the information gain. They can result in different tree structures.

splitter - default evaluates all features and chooses the split that maximizes the chosen criterion ("gini" or "entropy").

**max_depth: [3, 10, 20, 30]** last tree was 44 which is too much for 7 features..

**min_samples_split: [2, 5, 10]** Increasing min_samples_split forces the tree to make splits only when there are enough samples, preventing it from creating overly specific branches that might fit noise in the training data.

**min_samples_leaf: [1, 2, 5]** increasing min_samples_leaf forces the tree to create larger, more generalized leaf nodes, which can improve performance on unseen data.

min_weight_fraction_leaf -  minimum weighted fraction of the sum total of weights (of all the input samples) required to be at a leaf node.

**max_features: [None, 5, 3]** None = all. log(7) or sqrt(7) will be 3, and i chose 1 in between. Limiting max_features can sometimes improve performance by reducing overfitting

**max_leaf_nodes': [None, 2000, 3500, 5000, 6500]** tree had 56,000 leaf noedes which was too much.

**min_impurity_decrease': [0.0, 0.1, 0.2]** sets a minimum threshold for the impurity reduction. A split will only occur if the impurity reduction is greater than or equal to this threshold. It helps prevent the tree from making splits that result in very small impurity reductions, which might indicate overfitting.  

class_weight=None

**ccp_alpha': [0.0, 0.1, 0.2]** cost-complexity pruning in decision trees.
It controls the trade-off between the size of the tree and its accuracy on the training data.   
Higher ccp_alpha values lead to more pruning, resulting in smaller trees
<img src="images/ccp_alpha.png" width=400 height=300>

monotonic_cst=None Indicates the monotonicity constraint to enforce on each feature.


In [ ]:
X_train, X_test, y_train, y_test = get_train_test()

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': [None, 5, 3],
    'max_leaf_nodes': [None, 2000, 3500, 5000, 6500],
    'min_impurity_decrease': [0.0, 0.1, 0.2],
    'ccp_alpha': [0.0, 0.1, 0.2]
}

grid_search = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=param_grid, cv=5)

grid_search.fit(X_train, y_train)

dt_best_model = grid_search.best_estimator_

evaluate_model(dt_best_model, X_train, X_test, y_train, y_test)
print_decision_tree_params(dt_best_model)

156 minutes vs 16 seconds same accuracy.
only needed to add max leaf nodes

In [ ]:
X_train, X_test, y_train, y_test = get_train_test()

param_grid = {
    'max_leaf_nodes': [None, 2000, 3500, 5000, 6500],
}

grid_search_2 = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=param_grid, cv=5)

grid_search_2.fit(X_train, y_train)

dt_best_model_2 = grid_search_2.best_estimator_

evaluate_model(dt_best_model_2, X_train, X_test, y_train, y_test)
print_decision_tree_params(dt_best_model_2)

In [ ]:
plt.figure(figsize=(20, 10))  # Adjust figure size as needed
plot_tree(dt_classifier, filled=True, feature_names=[f"feature_{i}" for i in range(X_train.shape[1])], class_names=[str(c) for c in dt_classifier.classes_]) # generate feature names, and class names
plt.show()

#### Random forest

##### Default Parameters

In [ ]:
X_train, X_test, y_train, y_test = get_train_test()

rf_classifier = RandomForestClassifier() 

rf_classifier.fit(X_train, y_train)

evaluate_model(rf_classifier, X_train, X_test, y_train, y_test)
print_random_forest_params(rf_classifier)

##### HyperParameter Tuninig

In [ ]:
param_grid = {
     'max_depth': [None, 3, 7, 15],  # Maximum depth of the tree
}

rf = RandomForestClassifier(random_state=42) #Setting a random state for reproducibility

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

grid_search.fit(X_train, y_train)

rf_best_model = grid_search.best_estimator_
evaluate_model(rf_best_model, X_train, X_test, y_train, y_test)
print_random_forest_params(rf_best_model)

#### XGBoost

##### Default Params

In [ ]:
X_train, X_test, y_train, y_test = get_xgboost_train_test_semi_categorical()

xgb_model = XGBClassifier(objective='multi:softprob', num_class=4, random_state=42, enable_categorical=True) # multi class returns propabilites
xgb_model.fit(X_train, y_train)

evaluate_model(xgb_model, X_train, X_test, y_train, y_test)
print_xgb_params(xgb_model)

In [ ]:
# try with everything categorical

X_train, X_test, y_train, y_test = get_xgboost_train_test_all_categorical()

xgb_model = XGBClassifier(objective='multi:softprob', num_class=4, random_state=42, enable_categorical=True) # multi class returns propabilites
xgb_model.fit(X_train, y_train)

evaluate_model(xgb_model, X_train, X_test, y_train, y_test)
print_xgb_params(xgb_model)

##### HyperParameter Tuninig

XGBClassifier( n_estimators: int = 1, *, booster: typing.Literal["gbtree", "dart"] = "gbtree", dart_normalized_type: typing.Literal["tree", "forest"] = "tree", tree_method: typing.Literal["auto", "exact", "approx", "hist"] = "auto", min_tree_child_weight: int = 1, colsample_bytree: float = 1.0, colsample_bylevel: float = 1.0, colsample_bynode: float = 1.0, gamma: float = 0.0, max_depth: int = 6, subsample: float = 1.0, reg_alpha: float = 0.0, reg_lambda: float = 1.0, learning_rate: float = 0.3, max_iterations: int = 20, tol: float = 0.01, enable_global_explain: bool = False, xgboost_version: typing.Literal["0.9", "1.1"] = "0.9" )

n_estimators - num_parallel_tree

booster - gbtree (gradient boosting with trees) is generally a stronger choice for low-dimensional data. no need for  dart (Dropout Additive Regression Trees) and much slower

tree_method - (Tree Construction Algorithms) didnt look better.

min_child_weight: minimum sum of instance weight (hessian) needed in a child. This can be used to control the complexity of the decision tree by preventing the creation of too small leaves. tried [1, 5, 10] and didnt improve.

colsample_bytree: percentage of columns used for each tree construction. Lowering this value can prevent overfitting by training on a subset of the features. we dont have overfitting, and when we trained 1 tree to take all features was best.
same for by_tree and by_node

gamma: minimum loss reduction required to make a further partition on a leaf node of the tree. Higher values increase the regularization. 

**max_depth: [4, 6, 8]** Deeper trees can capture more complex patterns in the data, but may also lead to overfitting.

subsample: percentage of rows used for each tree construction. Lowering this value can prevent overfitting by training on a smaller subset of the data.

reg_alpha	- L1 regularization term on weights (xgb's alpha). Default to 0.0.

reg_lambda- L2 regularization term on weights (xgb's lambda). Default to 1.0.

**learning_rate: [0.1, 0.3, 0.4]** deafulat 0.3

 max_iterations - would be redundant because early stopping already determines the optimal number of boosting rounds based on data and other hyperparameters.

 tol - Minimum relative loss improvement necessary to continue training


In [ ]:
X_train, X_test, y_train, y_test = get_xgboost_train_test_semi_categorical()

xgb_model = XGBClassifier(objective='multi:softprob', num_class=4, random_state=42, enable_categorical=True)

param_grid = {
    'max_depth': [4, 6, 8, 12],
    'learning_rate': [0.05, 0.08, 0.1, 0.3, 0.4],
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

xgb_best_model = grid_search.best_estimator_
evaluate_model(xgb_best_model, X_train, X_test, y_train, y_test)
print_xgb_params(xgb_best_model)

In [ ]:
X_train, X_test, y_train, y_test = get_xgboost_train_test_all_categorical()

xgb_model = XGBClassifier(objective='multi:softprob', num_class=4, random_state=42, enable_categorical=True)

param_grid = {
    'max_depth': [4, 6, 8, 12],
    'learning_rate': [0.05, 0.08, 0.1, 0.3, 0.4],
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

xgb_best_model = grid_search.best_estimator_
evaluate_model(xgb_best_model, X_train, X_test, y_train, y_test)
print_xgb_params(xgb_best_model)

### 2.C) Results Overview

| Model | Accuracy |
|---|---|
| Naive Bayes | 0.34 |
| Decision Tree | 0.42 |
| Random Forest | 0.37 |
| XGBoost | 0.42 |

The fact that only a single decision tree surpasssed random forest and XGBoost was surprising.

**Observations:**
* **Decision Tree and XGBoost:** Both Decision Tree and XGBoost achieved the highest accuracy of 0.42. This suggests that these models were able to learn more complex patterns in the data compared to Naive Bayes and Random Forest.
* **Naive Bayes:** Naive Bayes had the lowest accuracy at 0.34. This might indicate that the assumption of feature independence made by Naive Bayes doesn't hold well for your data.


We couldnt surpass 42%. <br>

### 2.D) Most Effective Parameters

The most effective parameters as we explained before were max_depth - because the tree grew too large becuase we have a lot of data. also max_leaf_nodes which capped the tree size did a good job - accuracy went up by 15% and reduced overfitting.

### 2.E) Statistical Anylsis

In [ ]:
#evaluate_models(nb_best_model, dt_best_model_2, rf_best_model, xgb_best_model)

def evaluate_multiclass_model(model, X_test, y_test, model_name, n_classes):
    # Predict probabilities for ROC curve
    y_prob = model.predict_proba(X_test)
    
    # Binarize the output for ROC
    y_test_bin = label_binarize(y_test, classes=range(n_classes))
    
    # Initialize dictionaries to store metrics
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    # Compute ROC curve and ROC area for each class
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_prob.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    # Predict labels for other metrics
    y_pred = model.predict(X_test)
    
    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    
    # Calculate per-class FPR and FNR
    fpr_per_class = []
    fnr_per_class = []
    for i in range(n_classes):
        tn = np.sum(cm[:, :i].sum() + cm[:, i+1:].sum())  # True Negatives for class i
        fp = np.sum(cm[:, i]) - cm[i, i]  # False Positives for class i
        fn = cm[i, :].sum() - cm[i, i]  # False Negatives for class i
        tp = cm[i, i]  # True Positives for class i
        fpr_per_class.append(fp / (fp + tn) if (fp + tn) > 0 else 0)
        fnr_per_class.append(fn / (fn + tp) if (fn + tp) > 0 else 0)
    
    # Compute global metrics (macro average)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Print results
    print(f"\nMetrics for {model_name}:")
    print(f"ROC AUC (Micro): {roc_auc['micro']:.4f}")
    print(f"Per-class ROC AUC: {[f'{i}: {roc_auc[i]:.4f}' for i in range(n_classes)]}")
    print(f"Per-class FPR: {[f'{i}: {fpr_per_class[i]:.4f}' for i in range(n_classes)]}")
    print(f"Per-class FNR: {[f'{i}: {fnr_per_class[i]:.4f}' for i in range(n_classes)]}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (Macro): {precision:.4f}")
    print(f"Recall (Macro): {recall:.4f}")
    print(f"F1 Score (Macro): {f1:.4f}")
    
    return fpr, tpr, roc_auc, y_pred, cm

models = {
    "best_nb": nb_best_model,
    "best_dt": dt_best_model_2,
    "best_rf": rf_best_model,
    "best_xgb": xgb_best_model,
}

results = {}
for name, model in models.items():
    fpr, tpr, roc_auc, y_pred, cm = evaluate_multiclass_model(model, X_test, y_test, name, 4)
    results[name] = {"fpr": fpr, "tpr": tpr, "roc_auc": roc_auc, "y_pred": y_pred, "cm": cm}

# Plot ROC curves for all models (micro-average)
plt.figure(figsize=(8, 6))
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red'])
for name, result in results.items():
    plt.plot(result["fpr"]["micro"], result["tpr"]["micro"], 
             color=next(colors), label=f'{name} (AUC = {result["roc_auc"]["micro"]:.2f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Micro-average ROC Curve Comparison (Multiclass)')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

# Optional: Print a summary table (macro averages)
print("\nSummary Table:")
print("Model     | AUC (Micro) | Accuracy | Precision | Recall | F1 Score")
print("-" * 70)
for name, result in results.items():
    y_pred = result["y_pred"]
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    print(f"{name:9} | {result['roc_auc']['micro']:.4f} | {accuracy:.4f} | {precision:.4f} | {recall:.4f} | {f1:.4f}")

## Step 3 - Clustering Based on Price Polices 


### 3.6 Identify the 150 hotels with the most data in the dataset and extract their records.


In [ ]:
import pandas as pd

file_path = "./hotels_data_changed.csv"  
df = pd.read_csv(file_path)

hotel_counts = df['Hotel Name'].value_counts()
top_150_hotels = hotel_counts.head(150).index
filtered_df = df[df['Hotel Name'].isin(top_150_hotels)]

display(filtered_df)

### 3.7 Find the 40 most common check-in dates  in the dataset and extract their records.


In [ ]:
checkin_counts = filtered_df['Checkin Date'].value_counts()
top_40_checkin_dates = checkin_counts.head(40).index
filtered_checkin_df = filtered_df[filtered_df['Checkin Date'].isin(top_40_checkin_dates)]

display(filtered_checkin_df)

### 3.8 160-dimensional feature vector

**Task**

Build a 160-dimensional feature vector for each hotel based on its discount pricing behavior. Each vector is constructed by:
- Filtering the top 150 hotels (by record count) and the top 40 checkin dates.
- For each hotel, extracting 4 discount prices (one per discount code) for each of the 40 checkin dates. If several snapshots exist, we will select the one's with minimal prices.
- If no data is available for a specific (checkin date, discount code) combination, mark it with `-1`.

**Plan**

1. **Group the Data:**  
   Group the filtered data by **Hotel Name**, **Checkin Date**, and **Discount Code**. For each group, compute the minimum discount price, ensuring that only the best (lowest) price per combination is selected.

2. **Pivot to Wide Format:**  
   Transform the grouped data into a wide format where:
   - Each row represents a single hotel.
   - Each column represents a unique (Checkin Date, Discount Code) combination, totaling 160 columns (40 dates × 4 codes).

3. **Fill Missing Data:**  
   - Reindex the pivoted DataFrame so that every hotel has all 160 combinations, filling missing entries with `-1`.

In [ ]:
# 1. Group by Hotel Name, Checkin Date, and Discount Code and select the minimum Discount Price.
grouped = (
    filtered_checkin_df
    .groupby(['Hotel Name', 'Checkin Date', 'Discount Code'])['Discount Price']
    .min()
    .reset_index()
)

# 2. Pivot the DataFrame so that:
#    - The index is 'Hotel Name'
#    - The columns are a MultiIndex with levels (Checkin Date, Discount Code)
#    - The values are the minimum discount prices.
pivot_df = grouped.pivot_table(index='Hotel Name',
                               columns=['Checkin Date', 'Discount Code'],
                               values='Discount Price')


# 3. Reindex the columns so that all 40 checkin dates and 4 discount codes are present.
#    Use the top_40_checkin_dates (from your earlier filtering) and the list [1, 2, 3, 4] for discount codes. 
all_combinations = pd.MultiIndex.from_product([top_40_checkin_dates, [1, 2, 3, 4]],
                                                names=['Checkin Date', 'Discount Code'])

pivot_df = pivot_df.reindex(columns=all_combinations, fill_value=-1)
pivot_df = pivot_df.fillna(-1)


pivot_df.columns = [
    col if isinstance(col, str) else f"{col[0]} - {col[1]}"
    for col in pivot_df.columns
]
pivot_df = pivot_df.reset_index()

print(pivot_df.shape[0]) # Note we have 149 hotels instead of 150 - solution in next cell
display(pivot_df)

#### Identifying Missing Hotel

**Verifying Missing Hotel Data**

After filtering and pivoting the data, we expect to have 150 hotels, but only 149 appear in our pivot table. This indicates that one (or more) of the top 150 hotels has no records for the top 40 check-in dates used in our analysis.

The code above does the following:
1. **Identify Missing Hotels:**  
   It compares the complete list of top 150 hotels (`top_150_hotels`) with the hotel names present in the pivoted DataFrame (`pivot_df`). Any hotel that is not present is added to the `missing_hotels` list.

2. **Check Data for Each Missing Hotel:**  
   For each missing hotel, it filters `filtered_checkin_df` (which already contains only records from the top 40 check-in dates) to see if there are any records for that hotel.  
   - If the resulting DataFrame is empty, it confirms that the hotel indeed has no data for those check-in dates.  
   - This explains why the hotel did not appear in the pivot table.

By verifying that the missing hotel has no records in the filtered data, we can conclude that the drop in the number of hotels is due to the absence of data for those check-in dates rather than an error in our processing pipeline.



In [ ]:
# Assuming you have already defined:
# - top_150_hotels: the complete list of top 150 hotel names.
# - pivot_df: the pivoted DataFrame after grouping and filtering.
# - filtered_checkin_df: the DataFrame filtered by top 40 check-in dates.
#
# And the missing hotels are identified as:
missing_hotels = [hotel for hotel in top_150_hotels if hotel not in pivot_df['Hotel Name'].values]
print("Missing hotels:", missing_hotels)

# For each missing hotel, check if there is any record in the filtered_checkin_df.
for hotel in missing_hotels:
    hotel_records = filtered_checkin_df[filtered_checkin_df['Hotel Name'] == hotel]
    print(f"\nRecords for missing hotel '{hotel}':")
    print(hotel_records)  # This should print an empty DataFrame if no data is present.


### 3.9 Normalize 0-100


**Task**

For each hotel, we have a 160-dimensional vector of discount prices (one for each combination of Checkin Date and Discount Code). The goal is to normalize these prices so that, for each hotel, the lowest valid discount price becomes 0 and the highest becomes 100. Any missing value (indicated by `-1`) should remain unchanged.

**Plan**

1. **Define a Normalization Function:**  
   Create a function (`normalize_row`) that:
   - Filters out the missing values (`-1`) from the row.
   - Computes the minimum and maximum values among the valid discount prices.
   - Applies the normalization formula:
     $$
     \text{normalized\_price} = \frac{(\text{price} - \text{min\_price})}{(\text{max\_price} - \text{min\_price})} \times 100
     $$
   - Handles the case where all valid prices are equal (to avoid division by zero) by setting them to 0.

2. **Apply the Function Row-wise:**  
   Normalize the discount prices for each hotel (i.e., for each row) by applying the function to all columns except the "Hotel Name".

3. **Round and Convert to Integers:**  
   After normalization, round the values to the nearest integer and convert them to an integer type, ensuring that the normalized prices are stored as integers.



In [ ]:
def normalize_row(row):
    valid_mask = row != -1
    valid_prices = row[valid_mask]
    
    if valid_prices.empty:
        return row
    
    min_price = valid_prices.min()
    max_price = valid_prices.max()
    
   # Avoid division by zero if all valid prices are identical
    if min_price == max_price:
        row[valid_mask] = 0
    else:
        # Compute the normalized values, round them, and cast to int
        normalized_values = ((row[valid_mask] - min_price) / (max_price - min_price)) * 100
        row[valid_mask] = normalized_values.round(0).astype(int)
    
    return row


pivot_df.iloc[:, 1:] = pivot_df.iloc[:, 1:].apply(normalize_row, axis=1)

for col in pivot_df.columns[1:]:
    pivot_df[col] = pd.to_numeric(pivot_df[col], errors='coerce')
    pivot_df[col] = pivot_df[col].astype("Int64")


display(pivot_df)

### 3.10 Save to CSV

In [ ]:
hotels_clustering_data = "./hotels_clustering_data.csv"
pivot_df.to_csv(hotels_clustering_data, index=False)

### 3.11 Hierarchical Clustering

**Task**

Using the normalized discount prices for each hotel, we will perform hierarchical clustering to group hotels that exhibit similar pricing behaviors. We have a 160-dimensional feature vector for each hotel (each dimension corresponds to a specific (Checkin Date, Discount Code) pair).

**Plan**

1. **Prepare the Data:**  
   - Load the saved CSV file (`hotels_clustering_data.csv`).
   - Separate the "Hotel Name" column (for labeling) from the numeric feature columns.

2. **Perform Hierarchical Clustering:**  
   - Use SciPy's `linkage` function with Ward's method (which works well with Euclidean distance) to compute the clustering.
   - Generate a linkage matrix that represents the hierarchical clustering.

3. **Plot the Dendrogram:**  
   - Use SciPy's `dendrogram` function to visualize the hierarchical clustering.
   - Label each leaf in the dendrogram with the corresponding hotel name to help interpret the clusters.


In [ ]:
%pip install plotly

In [ ]:
import pandas as pd
import plotly.figure_factory as ff
from scipy.cluster.hierarchy import linkage

def create_dendrogram_from_csv(csv_path, color_threshold=825, width=1200, height=1800):

    clu_df = pd.read_csv(csv_path)
    hotel_names = clu_df["Hotel Name"].values
    X = clu_df.drop("Hotel Name", axis=1).values
    Z = linkage(X, method='ward')

    fig = ff.create_dendrogram(
        X,
        orientation='left',
        labels=hotel_names,
        color_threshold=color_threshold,
        linkagefun=lambda x: Z
    )

    fig.update_layout(
        width=width,
        height=height
    )
    fig.show()
    
    return fig

color_threshold_list = [825, 750, 625, 500]
for color_threshold in color_threshold_list:
    print (f'color_threshold={color_threshold}')
    create_dendrogram_from_csv("hotels_clustering_data.csv", color_threshold=color_threshold)


##### Results analysis

We performed hierarchical clustering on a dataset of hotels, where each hotel is represented by a **160-dimensional vector** of normalized discount prices. In simpler terms, each hotel’s vector shows *how* it discounts (and by how much) across different dates and discount codes. The dendrogram below clusters these hotels based on their similarity in discounting patterns.

Below, we examine **five different “cut” distances**—825, 750, 625, 500, and an additional view with fewer, broader clusters—and describe the cluster/subgroup formations you see in each figure.

---

#### Overall Explanation of the Dendrogram
- **X-axis**: The distance (or dissimilarity) at which clusters merge. Larger values mean more dissimilar groups.  
- **Y-axis**: The list of hotels, labeled along the left side.  
- **Colored Branches**: Each color indicates a cluster or subgroup under the specified distance threshold.

In general:  
- Hotels that **merge at smaller distances** (farther to the left in the dendrogram) are quite similar in how they price their discounts.  
- If you follow the dendrogram to the right until a major branch merges, that indicates hotels (or clusters of hotels) that are more dissimilar in their pricing behavior.

---

#### 1. Cut at Distance ≈ 825

![Cut at ~825](./images/clustering-825.png)  

- We see **three main clusters** forming at this high distance threshold.  
- **Group 1** (green), **Group 2** (red), and **Group 3** (blue) represent broad differences in discounting strategies. 
- Within each group, hotels share overall similarities in their discount patterns, but we’re not yet seeing the finer differences.

---

#### 2. Cut at Distance ≈ 750

![Cut at ~750](./images/clustering-750.png)  

- Lowering the threshold to **~750** begins to reveal **subgroups** within **Groups 1** and **3**, while **Group 2** remains mostly intact.  
- **Group 1** (previously green) starts splitting into multiple smaller clusters. These subgroups suggest that, while these hotels share a broad discount pattern, some differences in exact pricing behavior are now visible.  
- **Group 3** also subdivides more than before.
- **Group 2** is still one cluster, which indicates that these hotels are, as a whole, more cohesive or less varied in their discount strategies at this distance.

---

#### 3. Cut at Distance ≈ 625

![Cut at ~625](./images/clustering-625.png)  

- By **~625**, **Group 1** and **Group 3** break into several distinct subgroups, illustrating more granular differences.  
- **Group 2** is still a single cluster—indicating even at this more fine-grained level, the hotels in Group 2 remain very similar.  
- This suggests **Group 2** has a stable, uniform discount pattern, or at least they differ less from each other than from the other groups.

---

#### 4. Cut at Distance ≈ 500

![Cut at ~500](./images/clustering-500.png)  

- Finally, at **~500**, **Group 2** splits into **two subgroups**, showing that there are *at least* two distinct discounting patterns within Group 2 when we look closely.  
- **Groups 1 and 3** now fragment into many smaller subgroups. If you count them, there could be several mini-clusters in each.  
- Each **subgroup** here represents hotels that are very closely aligned in how they handle discounts across time—likely direct competitors or hotels following near-identical pricing guidelines.

---

#### Meaning of Subgroups

1. **Competitive Landscape**  
   - Hotels in the **same subgroup** are likely *direct competitors*, as they share nearly identical discount structures and timelines.

2. **Revenue Management Strategy**  
   - Subgroups often align with brand or chain policies (e.g., a single chain might appear as a tight cluster if they use centralized pricing software).  
   - The level of granularity around **500** distance reveals the *very fine details* of each hotel’s promotional activities.

3. **Marketing & Differentiation**  
   - If you manage a hotel in these clusters, you might examine how close neighbors in the dendrogram are discounting.  
   - Being in a tight cluster might motivate you to **differentiate** or **align** your pricing further.

4. **Insights for Group 2**  
   - Group 2’s stability up to a lower threshold (625) suggests a coherent pricing approach among its hotels. They only start splitting at ~500, which indicates that their differences are more subtle and only become apparent under a finer lens.

---

#### Putting It All Together
At **higher thresholds** (825+), we see broad strokes: *three main pricing behaviors*.  As we lower the cut distance, we discover **finer distinctions** within each major group:
- **Groups 1 & 3** develop several distinct subgroups earlier (around 750 or 625).  
- **Group 2** remains consistent longer and only splits around 500, revealing that even seemingly uniform clusters can exhibit small internal differences if you zoom in far enough.

In essence, **each subgroup** helps us understand small sets of hotels that might share very close discounting patterns—key information for competitive analysis and strategic pricing decisions.



#### diving even deeper - (checking the hotel stars and average price with the pricing strategy)

The results from the last section were interesting, so i decided to run the same clustering but with the hotels stars, prices and discounts to see if some patterns emerge.

The new label now contains:
- stars
- avg price
- avg discount
- avg discount rate 

in this format:

`(stars) - price - discount - discount rate`

example:

(5) - 3898 - 3663 - 6% 

In [ ]:
import pandas as pd

pivot_df = pd.read_csv("hotels_clustering_data.csv")
df = pd.read_csv("./hotels_data_changed.csv")

pivot_df["Hotel Name"] = pivot_df["Hotel Name"].astype(str).str.strip()
hotel_counts = df["Hotel Name"].value_counts()
top_150_hotels = hotel_counts.head(150).index

summary_df = (
    df[df["Hotel Name"].isin(top_150_hotels)]
    .groupby("Hotel Name")
    .agg({"Original Price": "mean", "Discount Price": "mean", "Hotel Stars": "first"})
    .reset_index()
)

summary_df["Hotel Name"] = summary_df["Hotel Name"].astype(str).str.strip()
summary_df["Original Price"] = summary_df["Original Price"].round(0).astype(int)
summary_df["Discount Price"] = summary_df["Discount Price"].round(0).astype(int)
merged_df = pivot_df.merge(summary_df, on="Hotel Name", how="left")

merged_df["DiscountPerc"] = (((merged_df["Original Price"] - merged_df["Discount Price"]) / merged_df["Original Price"]) * 100).round(0).astype(int)
merged_df["Label"] = merged_df.apply(lambda row: f"({row['Hotel Stars']}) - {row['Original Price']} - {row['Discount Price']} - {row['DiscountPerc']}%", axis=1)

merged_df.head()
merged_df.to_csv("./hotels_clustering_data_with_summary.csv", index=False)


In [ ]:
import pandas as pd
import plotly.figure_factory as ff
from scipy.cluster.hierarchy import linkage

def create_dendrogram_from_csv(csv_path, color_threshold=825, width=1200, height=1800):
    clu_df = pd.read_csv(csv_path)
    if "Label" in clu_df.columns:
        labels = clu_df["Label"].values
        non_clustering = ["Hotel Name", "Label", "Hotel Stars", "Original Price", "Discount Price", "DiscountPerc"]
    else:
        labels = clu_df["Hotel Name"].values
        non_clustering = ["Hotel Name", "Hotel Stars", "Original Price", "Discount Price", "DiscountPerc"]
    X = clu_df.drop(columns=non_clustering, errors='ignore').values
    Z = linkage(X, method='ward')
    fig = ff.create_dendrogram(
        X,
        orientation='left',
        labels=labels,
        color_threshold=color_threshold,
        linkagefun=lambda x: Z
    )
    fig.update_layout(width=width, height=height)
    fig.show()
    return fig

color_threshold_list = [825, 750, 625, 500]
for color_threshold in color_threshold_list:
    print(f'color_threshold={color_threshold}')
    create_dendrogram_from_csv("hotels_clustering_data_with_summary.csv", color_threshold=color_threshold)


#### Hierarchical Clustering with Star Rating, Avg Price, and Discounts

We re-ran clustering with four features:
1. **Hotel star rating**  
2. **Avg nightly price**  
3. **Avg absolute discount**  
4. **Avg discount rate (%)**

Each dendrogram label is `(stars) – price – discount – discount rate`.

#### 1. Main Observations

- **Star Rating & Price** dominate the first major splits:
  - **Budget/Midscale** (2–3★, ~1,200–1,800 price) form one big group.
  - **Upscale/Luxury** (4–5★, 2,500–4,000+) cluster separately.
- **Discount Patterns** refine clusters within each star tier:
  - Hotels sharing similar price but different discount behavior split into distinct subgroups.
  - Aggressive vs. conservative discounters separate at lower distance thresholds.

#### 2. Cluster Insights

1. **Large Distance Clusters**:
   - Budget vs. Luxury hotels split clearly by base price and star rating.
   - Some **mixed star** groups appear if their price/discount overlap.

2. **Subgroups at Tighter Distances**:
   - Within the same star tier, variations in discount size or percentage form separate sub-clusters.
   - Outliers can include a high-star hotel with heavy discounts or a low-star with high pricing.

#### 3. Takeaways

- **Star Rating is Primary**; discount strategies refine subgroups.  
- Hotels in tight clusters likely share near-identical pricing/discount policies, suggesting **direct competition**.  
- Examine outliers for unique pricing approaches or potential repositioning strategies.

## step 4 - Building a Predictive Model for Hotel Pricing Dynamics Using Snapshot Data (Deprecated - Task was unclear)



### Problem Breakdown



- **Objective:**  
  Predict the `Discount Price` for check-in dates within 30 days of a given `Snapshot Date` while minimizing the number of price samples (queries) from the database.

- **Context:**  
  Given a dataset with fields like `Snapshot Date`, `Hotel Name`, `Discount Code`, and various price-related features, our task is to build a model that can infer the price trends and predict missing prices accurately.

- **Performance Measure:**  
  The model's success is evaluated by the R-squared metric on a subset of 40 hotels with the most complete data.


### Solution Approaches: Gaussian Process Regression vs. Bayesian Linear and Polynomial Regression



#### Gaussian Process Regression



- **Pros:**
  - **Uncertainty Quantification:** Provides both predictions and confidence intervals, which is key for active sampling.
  - **Sample Efficiency:** Excels when data is limited, helping us select the most informative samples.
  - **Flexibility:** Capable of modeling complex, non-linear relationships inherent in price trends.
  
- **Cons:**
  - **Computational Cost:** Can become computationally expensive with larger datasets.
  - **Kernel Selection:** Requires careful tuning of the kernel functions to capture the underlying data patterns accurately.

- **Fit to the Problem:**  
  Ideal for our task since its uncertainty estimates allow us to strategically choose the next check-in dates to sample, ensuring we use as few queries as possible while maintaining high prediction performance.




#### Bayesian Linear and Polynomial Regression



- **Pros:**
  - **Simplicity:** Easy to implement and interpret.
  - **Speed:** Generally faster to train compared to GP, which can be advantageous with simpler or smooth trends.
  
- **Cons:**
  - **Limited Flexibility:** May not capture complex non-linear patterns as effectively.
  - **Uncertainty Estimates:** While Bayesian methods provide uncertainty measures, they are often less robust in capturing the variability in complex datasets compared to GP.

- **Fit to the Problem:**  
  Suitable if the price trends were very smooth and predictable. However, the nuances in hotel pricing (e.g., varying discount strategies, day-of-week effects) suggest a need for a more flexible approach.




#### Chosen Approach:



- **Gaussian Process Regression** is selected as the primary method. Its ability to provide uncertainty estimates and guide active sampling makes it the best fit for minimizing the number of price queries while still achieving a high R-squared on the test set.


### Feature Engineering and Data Preparation


#### Feature Engineering 

- **Date Encoding:**  
  Convert `Snapshot Date` and `Checkin Date` into numerical features (e.g., days from the snapshot, day-of-week, etc.) to capture temporal patterns.

- **Handling Categorical Data:**  
  Apply one-hot encoding or similar methods to features such as `Hotel Name` and `Discount Code`.

In [ ]:
data_path = "./hotels_data_changed.csv"
data = pd.read_csv(data_path)

data['OG_Hotel_Name'] = data['Hotel Name']

data = feature_engineering(data)
cols_to_drop = ['Snapshot Date', 'Checkin Date', 'WeekDay']
data = data.drop(columns=cols_to_drop)

# For our modeling purposes, we want to encode categorical variables.
# Here, we choose 'Hotel Name' and 'Discount Code' as categorical features.
categorical_features = ['Hotel Name', 'Discount Code']
data = encode_categorical_columns(data, categorical_features)


print(data.head(1))

#### Normalization/Scaling

Scale continuous features as needed to ensure all features contribute equally to the model training.

In [ ]:

features_to_scale = [
    'Snapshot_Date_num',  
    'Checkin_Date_num',   
    'Original Price',
    'Discount Price',
    'DiscountDiff',
    'DiscountPerc',
    'Available Rooms',
    'DayDiff'
]

# Normalize the data
normalized_data, fitted_scaler = normalize_features(data, features_to_scale)
normalized_data[features_to_scale].head()

#### Data split (Train/Test)



 Split the data into train and test Datasets.

In [ ]:
hotel_counts = normalized_data['OG_Hotel_Name'].value_counts()
top_40_hotels = hotel_counts.head(40).index.tolist()



test_data = normalized_data[normalized_data['OG_Hotel_Name'].isin(top_40_hotels)]
train_data = normalized_data[~normalized_data['OG_Hotel_Name'].isin(top_40_hotels)]

test_data = test_data.drop(columns=['OG_Hotel_Name'])
train_data = train_data.drop(columns=['OG_Hotel_Name'])

target_variable = 'Discount Price'

X_test = test_data.drop(columns=[target_variable])
y_test = test_data[target_variable]

X_train = train_data.drop(columns=[target_variable])
y_train = train_data[target_variable]


print("Test set (features):", X_test.shape)
print("Test set (target):", y_test.shape)
print("Training set (features):", X_train.shape)
print("Training set (target):", y_train.shape)


### Gaussian Process Regression: Training, Testing, and Approach Overview



In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.metrics import r2_score

# Set random seed for reproducibility
np.random.seed(42)

initial_sample_size = 30
all_indices = np.array(X_train.index)
initial_sample_indices = np.random.choice(all_indices, size=initial_sample_size, replace=False)
sampled_indices = list(initial_sample_indices)
candidate_indices = list(set(all_indices) - set(sampled_indices))

# Define Gaussian Process Regression model with a basic kernel
kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5)

max_iterations = 50  # Maximum number of iterations (samples to add)
r2_scores = []

# --- Iterative Sampling ---
for i in range(max_iterations):
    # Train GP on current sampled data
    X_sampled = X_train.loc[sampled_indices]
    y_sampled = y_train.loc[sampled_indices]
    gp.fit(X_sampled, y_sampled)
    
    # Evaluate on the test set
    y_pred, sigma = gp.predict(X_test, return_std=True)
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)
    print(f"Iteration {i+1}: Samples used = {len(sampled_indices)}, Test R² = {r2:.3f}")
    
    # Break if no more candidates are left
    if not candidate_indices:
        break

    # Predict on candidate pool to obtain uncertainty (standard deviation)
    X_candidates = X_train.loc[candidate_indices]
    _, sigma_candidates = gp.predict(X_candidates, return_std=True)
    
    # Select the candidate with the maximum uncertainty
    max_uncertainty_idx = np.argmax(sigma_candidates)
    new_sample_idx = X_candidates.index[max_uncertainty_idx]
    
    # Add this candidate to our sampled set and remove from candidates
    sampled_indices.append(new_sample_idx)
    candidate_indices.remove(new_sample_idx)

print("Final number of samples used:", len(sampled_indices))


## step 4 - Building a Predictive Model for Hotel Pricing Dynamics Using Snapshot Data (V2)




- **Objective:**  
  Predict the `Discount Price` for check-in dates within 30 days of a given `Snapshot Date` while minimizing the number of price samples (queries) from the database.

- **Context:**  
  Given fields of `Snapshot Date`, `Hotel Name`, `Discount Code`, build a relevant data set to predict the `Discount Price` for check-in dates within 30 days of the `Snapshot Date`.

- **Performance Measure:**  
  The model's success is evaluated by the R-squared metric on a subset of 40 hotels with the most complete data.


### Solution Approaches: Gaussian Process Regression vs. Bayesian Linear and Polynomial Regression



#### Gaussian Process Regression



- **Pros:**
  - **Uncertainty Quantification:** Provides both predictions and confidence intervals, which is key for active sampling.
  - **Sample Efficiency:** Excels when data is limited, helping us select the most informative samples.
  - **Flexibility:** Capable of modeling complex, non-linear relationships inherent in price trends.
  
- **Cons:**
  - **Computational Cost:** Can become computationally expensive with larger datasets.
  - **Kernel Selection:** Requires careful tuning of the kernel functions to capture the underlying data patterns accurately.

- **Fit to the Problem:**  
  Ideal for our task since its uncertainty estimates allow us to strategically choose the next check-in dates to sample, ensuring we use as few queries as possible while maintaining high prediction performance.




#### Bayesian Linear and Polynomial Regression



- **Pros:**
  - **Simplicity:** Easy to implement and interpret.
  - **Speed:** Generally faster to train compared to GP, which can be advantageous with simpler or smooth trends.
  
- **Cons:**
  - **Limited Flexibility:** May not capture complex non-linear patterns as effectively.
  - **Uncertainty Estimates:** While Bayesian methods provide uncertainty measures, they are often less robust in capturing the variability in complex datasets compared to GP.

- **Fit to the Problem:**  
  Suitable if the price trends were very smooth and predictable. However, the nuances in hotel pricing (e.g., varying discount strategies, day-of-week effects) suggest a need for a more flexible approach.




#### Chosen Approach:



- **Gaussian Process Regression** is selected as the primary method. Its ability to provide uncertainty estimates and guide active sampling makes it the best fit for minimizing the number of price queries while still achieving a high R-squared on the test set.


### Data Loading and Extraction:

1. **Load the Data:**  
   We read the CSV file into a DataFrame and ensure that date columns are properly converted to datetime objects.

2. **Filter the Data:**  
   We filter the dataset to include only rows where `DayDiff` is 30 or less, meaning the check-in is within 30 days after the snapshot date.

3. **Group the Data:**  
   We group by `Snapshot Date`, `Hotel Name`, and `Discount Code`, and count the number of transactions (rows) in each group. This count is stored in the column `transaction_count`.

4. **Dataset: onePerHotel:**  
   - For each hotel, we sort by `transaction_count` in descending order and select the combination with the maximum count.  
   - Then, we sort these unique hotel rows by `transaction_count` and take the top 40.

5. **Dataset: mostData:**  
   - We simply sort all the grouped combinations by `transaction_count` in descending order and select the top 40 combinations.

6. **Extracting Parameters:**  
   For each dataset, we extract a list of dictionaries (or "params") containing `Snapshot Date`, `Hotel Name`, and `Discount Code`.


In [ ]:
import pandas as pd

data_path = "./hotels_data_changed.csv"
data = pd.read_csv(data_path)

# We are only interested in transactions with a check-in date within 30 days after the snapshot.
data_filtered = data[data['DayDiff'] <= 30].copy()

data_filtered['Snapshot Date'] = pd.to_datetime(data_filtered['Snapshot Date']).dt.normalize()

# We group by the combination of 'Snapshot Date', 'Hotel Name', and 'Discount Code', and count the number of transactions.
grouped = data_filtered.groupby(
    ['Snapshot Date', 'Hotel Name', 'Discount Code']
).size().reset_index(name='transaction_count')

# For each hotel, choose the combination with the highest transaction_count.
# First, sort within each hotel so that the highest count is on top,
# then group by 'Hotel Name' and take the first (best) row.
onePerHotel = (
    grouped.sort_values(['Hotel Name', 'transaction_count'], ascending=[True, False])
    .groupby('Hotel Name')
    .first()
    .reset_index()
)

onePerHotel = onePerHotel.sort_values('transaction_count', ascending=False).head(40)

mostData = grouped.sort_values('transaction_count', ascending=False).head(40)

params_onePerHotel = onePerHotel[['Snapshot Date', 'Hotel Name', 'Discount Code']].to_dict('records')
params_mostData   = mostData[['Snapshot Date', 'Hotel Name', 'Discount Code']].to_dict('records')

print("onePerHotel (Top 40 hotels with best combination per hotel):")
display(onePerHotel.head())

print("mostData (Top 40 combinations overall):")
display(mostData.head())

### Feature Engineering and Data Preparation 

In this section we would create a function that gets the `Snapshot Date`, `Hotel Name`, and `Discount Code` and returns all we need to start training the model.
This includes the following:
-  get relevant data from the dataset
-  apply feature engineering on that data
-  normalize and scale the data
-  Splitting into X and y & Creating the Wrapper Function

#### get the data 

In [ ]:
def load_relevant_data(params, data):
 
    snapshot_date = pd.to_datetime(params['Snapshot Date']).normalize()
    
    subset = data[
        (data['Snapshot Date'] == snapshot_date) &
        (data['Hotel Name'] == params['Hotel Name']) &
        (data['Discount Code'] == params['Discount Code']) &
        (data['DayDiff'] <= 30) 
    ]
    return subset

test_params = params_mostData[0]
relevant_subset = load_relevant_data(test_params, data_filtered)
print("Parameters:", test_params)
print("Relevant Data (first 5 rows):")
display(relevant_subset.head())


#### apply feature engineering

In this step, we transform our raw data into a format that is ready for model training. The goals are to:

- **Convert Date Information:**  
  We convert the date column `Checkin Date` into numerical values (Unix timestamps) so that the model can understand and work with temporal data.

- **Encode Weekday Information:**  
  We convert the weekday from a string (e.g., "Monday") into a numeric value using the helper function `convert_weekday_to_num`, storing it as `WeekDay_num`.

- **Remove Irrelevant or Constant Columns:**  
  After feature engineering, some columns become redundant or uninformative:
  - **`Hotel Name`:** Removed because the data is filtered for a specific hotel, or if used for filtering, the raw name is no longer needed.
  - **`Checkin Date`:** Once their numerical representations are extracted, the raw date values are no longer required.
  - **`WeekDay`:** We already have the numeric `WeekDay_num`, making the original string column redundant.
  - **`Days`:** This column is always the same (always 5), so it does not provide any variance or useful information for training.
  - **`Snapshot Date`:** Since the snapshot is the same across the subset, this column is constant and does not help differentiate between records.
  - **`Snapshot ID`:** This is also constant for a given snapshot and can be removed to reduce noise.

The removal of these columns helps to simplify our dataset and ensures that the model is trained only on features that provide meaningful variation.


In [ ]:
engineered_data = feature_engineering(relevant_subset.copy())

print("Engineered Data")
display(engineered_data.head())

#### normalize and scale the data

**What are we doing?**  
We use a `StandardScaler` to standardize the continuous features by removing the mean and scaling to unit variance. This process prevents features with larger numeric ranges from dominating the learning process.

**Why are we doing it?**  
Normalization is crucial in many machine learning algorithms—especially when using distance-based metrics or gradient descent—for faster convergence and improved performance.

**Features to Scale:**  
Based on our engineered dataset, we scale:
- `Checkin_Date_num` (numerical representation of the check-in date)  
- `Original Price`  
- `Discount Price`  
- `Available Rooms`  
- `DayDiff`  
- `DiscountDiff`  
- `DiscountPerc`

In [ ]:
features_to_scale = [
    "Checkin_Date_num", 
    "Original Price", 
    "Discount Price", 
    "Available Rooms", 
    "DayDiff", 
    "DiscountDiff", 
    "DiscountPerc"
]

normalized_data, fitted_scaler = normalize_features(engineered_data, features_to_scale)

print("Normalized Data")
display(normalized_data.head())


####  Splitting into X and y & Creating the Wrapper Function

In this step, we:

- Split the normalized data into features (`X`) and target (`y`). Here, `y` is the scaled "Discount Price".  
- Create a wrapper function that runs the entire preprocessing pipeline (from getting relevant data to normalization) and returns `X`, `y`, the fitted scaler, and a helper function to reverse the scaling for the target.  
- The helper function uses the scaling parameters for "Discount Price" (found in the fitted scaler) so that later we can convert predictions back to their original values.

> **Why Remove "Discount Price" from X?**  
> The target variable should not be present in the features. By dropping "Discount Price" from the normalized data, we ensure that `X` contains only the input features while `y` contains the target variable.

> **Reversing the Scaling:**  
> Since the scaler is fitted on multiple columns, we locate the index corresponding to "Discount Price" in our `features_to_scale` list and use its mean and scale to convert scaled predictions back to their original values.

In [ ]:
def prepare_training_data(params, data, sort=False):
    relevant = load_relevant_data(params, data)
    engineered = feature_engineering(relevant.copy())
    normalized, scaler = normalize_features(engineered, features_to_scale)
    
    if sorted:
        normalized = normalized.sort_values(by="Discount Price")

    X = normalized.drop(columns=["Discount Price"])
    y = normalized["Discount Price"]

    idx = features_to_scale.index("Discount Price")
    
    def reverse_scaling(y_scaled):
        return y_scaled * scaler.scale_[idx] + scaler.mean_[idx]
    
    return X, y, scaler, reverse_scaling

X, y, scaler, reverse_scaling = prepare_training_data(test_params,data_filtered )
print("X shape:", X.shape)
print("y shape:", y.shape)
print("First 5 values of y (scaled):", y.head().values)
print("First 5 values of y (original):", reverse_scaling(y.head().values))


### Gaussian Process Regression: Training And Testing



#### Model Initialization



In this cell, we initialize our GaussianProcessRegressor model. We use a kernel that combines a ConstantKernel and an RBF kernel. 

- **GaussianProcessRegressor:**  
  Provides both predictions and uncertainty estimates, which are essential for our active sampling approach.

- **ConstantKernel:**  
  Represents a constant bias in the function, serving as a baseline.

- **RBF Kernel (Radial Basis Function):**  
  Models smooth variations in the data, which is ideal for capturing trends in discount prices.



In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

def initialize_model():
    kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-2, normalize_y=True)
    return gp

model = initialize_model()
print("Initialized Model:")
print(model)


#### Active Sampling Loop


##### Maximum uncertainty based stop


In this cell, we implement the active sampling loop. The process is as follows:

- Start with an initial set of samples (e.g., the first, middle, and last indices).
- Iteratively fit the Gaussian Process model on the current training set.
- Predict on the remaining (unsampled) data and obtain uncertainty estimates.
- Select the candidate with the highest uncertainty and add it to the training set.
- Compute the `R²` score to monitor performance.
- Stop when the maximum uncertainty is below a threshold or when the maximum iterations are reached.



In [ ]:
import numpy as np
from sklearn.metrics import r2_score

def active_sampling_loop(X, y, max_iterations=10, uncertainty_threshold=0.05, initial_sample_indices=None):
    n_samples = X.shape[0]
    
    if initial_sample_indices is None:
        initial_sample_indices = [0, n_samples // 2, n_samples - 1]
    
    training_indices = set(initial_sample_indices)
    candidate_indices = set(range(n_samples)) - training_indices
    iteration_log = []
    
    for iteration in range(max_iterations):
        # Prepare current training data
        X_train = X.iloc[sorted(list(training_indices))]
        y_train = y.iloc[sorted(list(training_indices))]
        
        # Initialize and fit the Gaussian Process model
        model = initialize_model()
        model.fit(X_train, y_train)
        
        # Predict on candidate data and obtain uncertainties
        candidate_list = sorted(list(candidate_indices))
        if not candidate_list:
            break

        X_candidates = X.iloc[candidate_list]
        y_pred_candidates, std_candidates = model.predict(X_candidates, return_std=True)
        
        # Find candidate with maximum uncertainty
        max_std = np.max(std_candidates)
        chosen_candidate_idx = candidate_list[np.argmax(std_candidates)]
        
        y_pred_full = model.predict(X)
        current_r2 = r2_score(y, y_pred_full)
        iteration_log.append({
            'iteration': iteration,
            'num_samples': len(training_indices),
            'max_uncertainty': max_std,
            'r2': current_r2
        })
        
        
        if max_std < uncertainty_threshold:
            break
        
        # Add the most uncertain candidate to the training set
        training_indices.add(chosen_candidate_idx)
        candidate_indices.remove(chosen_candidate_idx)
    
    # Final training on the selected samples
    X_final = X.iloc[sorted(list(training_indices))]
    y_final = y.iloc[sorted(list(training_indices))]
    final_model = initialize_model()
    final_model.fit(X_final, y_final)
    final_predictions = final_model.predict(X)
    final_r2 = r2_score(y, final_predictions)
    
    return {
        'final_model': final_model,
        'final_predictions': final_predictions,
        'final_r2': final_r2,
        'iteration_log': iteration_log,
        'total_samples_used': len(training_indices)
    }

results = active_sampling_loop(X, y, max_iterations=15, uncertainty_threshold=0.05)

print("Final R² Score:", results['final_r2'])
print("Total Samples Used:", results['total_samples_used'])
print("Iteration Log:")
for log in results['iteration_log']:
    print(log)


##### Maximum uncertainty and stagnant Iterations based stop

In this cell we implement an enhanced active sampling loop. In addition to stopping when the maximum uncertainty is very low, we also monitor `R²` improvement. The loop will stop if either:
- The model's uncertainty is very low (`max_std < uncertainty_threshold`), or
- The uncertainty is moderately low (`max_std < higher_uncertainty_threshold`) **and** the `R²` improvement has been stagnant for a set number of iterations.


In [ ]:
def active_sampling_loop_with_stagnant_iterations_stop(X, y, max_iterations=10, 
                                                        uncertainty_threshold=0.05, 
                                                        higher_uncertainty_threshold=0.1,
                                                        initial_sample_indices=None, 
                                                        r2_improvement_threshold=1e-4, 
                                                        max_stagnant_iterations=3):
    n_samples = X.shape[0]
    if initial_sample_indices is None:
        initial_sample_indices = [0, n_samples // 2, n_samples - 1]
    
    training_indices = set(initial_sample_indices)
    candidate_indices = set(range(n_samples)) - training_indices
    iteration_log = []
    
    previous_r2 = -np.inf  
    stagnant_iterations = 0  
    
    for iteration in range(max_iterations):
        # Prepare training data for the current iteration
        X_train = X.iloc[sorted(list(training_indices))]
        y_train = y.iloc[sorted(list(training_indices))]
        
        # Initialize and fit the model
        model = initialize_model()
        model.fit(X_train, y_train)
        
        # Predict on candidate data to obtain uncertainties
        candidate_list = sorted(list(candidate_indices))
        if not candidate_list:
            break

        X_candidates = X.iloc[candidate_list]
        y_pred_candidates, std_candidates = model.predict(X_candidates, return_std=True)
        
        max_std = np.max(std_candidates)
        chosen_candidate_idx = candidate_list[np.argmax(std_candidates)]
        
        
        y_pred_full = model.predict(X)
        current_r2 = r2_score(y, y_pred_full)
        
        
        r2_improvement = current_r2 - previous_r2
        if r2_improvement < r2_improvement_threshold:
            stagnant_iterations += 1
        else:
            stagnant_iterations = 0
        
        previous_r2 = current_r2  
        
        iteration_log.append({
            'iteration': iteration,
            'num_samples': len(training_indices),
            'max_uncertainty': max_std,
            'r2': current_r2,
            'r2_improvement': r2_improvement
        })
        
        
        # Stop if the model is very confident OR if it is moderately confident and not improving.
        if (max_std < uncertainty_threshold) or ((max_std < higher_uncertainty_threshold) and (stagnant_iterations >= max_stagnant_iterations)):
            break
        
        # Add the candidate with highest uncertainty to the training set
        training_indices.add(chosen_candidate_idx)
        candidate_indices.remove(chosen_candidate_idx)
    
    # Final training on the selected samples
    X_final = X.iloc[sorted(list(training_indices))]
    y_final = y.iloc[sorted(list(training_indices))]
    final_model = initialize_model()
    final_model.fit(X_final, y_final)
    final_predictions = final_model.predict(X)
    final_r2 = r2_score(y, final_predictions)
    
    return {
        'final_model': final_model,
        'final_predictions': final_predictions,
        'final_r2': final_r2,
        'iteration_log': iteration_log,
        'total_samples_used': len(training_indices)
    }


results = active_sampling_loop_with_stagnant_iterations_stop(X, y, max_iterations=15, 
                                                              uncertainty_threshold=0.05, 
                                                              higher_uncertainty_threshold=0.1,
                                                              r2_improvement_threshold=1e-4, 
                                                              max_stagnant_iterations=3)

print("Final R² Score:", results['final_r2'])
print("Total Samples Used:", results['total_samples_used'])
print("Iteration Log:")
for log in results['iteration_log']:
    print(log)


#### Visualizing the Results



In this cell we visualize the active sampling process. The first part prints a log that shows, for each iteration, the number of samples used, the maximum uncertainty, and the R² score. The chart below shows two plots:
- **R² Evolution:** How the model's performance improves over iterations.
- **Max Uncertainty Evolution:** How the maximum prediction uncertainty decreases as more samples are added.


In [ ]:
import matplotlib.pyplot as plt

def print_iteration_log(results, total_data_count):
    final_samples = results['total_samples_used']
    percent_used = final_samples / total_data_count * 100
    print(f"Total Data Available: {total_data_count}")
    print(f"Total Samples Used: {final_samples} ({percent_used:.2f}%)")
    print("\nIteration Log:")
    for log in results['iteration_log']:
        print(f"Iteration {log['iteration']}: {log['num_samples']} samples, "
              f"Max Uncertainty: {log['max_uncertainty']:.4f}, R²: {log['r2']:.4f}")
    print(f"\nFinal R² Score: {results['final_r2']:.4f}")

def plot_active_sampling_results(results, total_data_count):
    iterations = [log['iteration'] for log in results['iteration_log']]
    r2_values = [log['r2'] for log in results['iteration_log']]
    uncertainties = [log['max_uncertainty'] for log in results['iteration_log']]
    
    final_samples = results['total_samples_used']
    percent_used = final_samples / total_data_count * 100

    fig, ax = plt.subplots(2, 1, figsize=(8, 10))
    
    ax[0].plot(iterations, r2_values, marker='o', linestyle='-')
    ax[0].set_title('R² Evolution Over Iterations')
    ax[0].set_xlabel('Iteration')
    ax[0].set_ylabel('R² Score')
    ax[0].grid(True)
    ax[0].text(0.5, 0.1, f'Final R²: {results["final_r2"]:.4f}', 
               transform=ax[0].transAxes, fontsize=12, color='green',
               bbox=dict(facecolor='white', alpha=0.8, edgecolor='green'))
    
    
    ax[1].plot(iterations, uncertainties, marker='o', linestyle='-')
    ax[1].set_title('Max Uncertainty Evolution Over Iterations')
    ax[1].set_xlabel('Iteration')
    ax[1].set_ylabel('Max Uncertainty (std)')
    ax[1].grid(True)
    
    plt.suptitle(f'Active Sampling: {final_samples} of {total_data_count} samples used ({percent_used:.2f}%)', fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

total_data_count = X.shape[0]  # For example, if X has 44 rows

print_iteration_log(results, total_data_count)
plot_active_sampling_results(results, total_data_count)


### Gaussian Process Regression - Full dataset Testing

#### Active Sampling Model Execution

In this section, we execute our active sampling models over all parameter combinations. We provide two variants:
- **Basic Active Sampling Loop:** The standard loop that iteratively selects samples based on maximum uncertainty.
- **Active Sampling Loop with Stagnant Iterations Stop:** An enhanced loop that also monitors `R²` improvements and stops if the model's performance stagnates while uncertainty is moderately low.

This section prepares the results for each parameter combination by running the appropriate active sampling function and collecting key metrics such as the final R², total samples used, and iteration logs.


In [ ]:
%%capture
%pip install tqdm

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
from tqdm import tqdm

def run_active_sampling_basic(params_list, data, sort=False, verbose=False, **hyperparams):
    results_list = []
    if not verbose:
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    for i, param in enumerate(tqdm(params_list, desc="Running basic active sampling", disable=verbose)):
        X, y, scaler, reverse_scaling = prepare_training_data(param, data, sort=sort)
        total_data_count = X.shape[0] 
        
        result = active_sampling_loop(X, y, **hyperparams)
        result['params'] = param
        result['total_data_count'] = total_data_count
        results_list.append(result)
        if verbose:
            print(f"Completed parameter {i+1}/{len(params_list)}: R² = {result['final_r2']:.4f}, Samples = {result['total_samples_used']} of {X.shape[0]}")
            
    warnings.filterwarnings("default", category=ConvergenceWarning)

    return results_list

basic_hyperparams = {
    "max_iterations": 15,
    "uncertainty_threshold": 0.05,
}

results_basic = run_active_sampling_basic(
params_mostData[:3], data_filtered, 
verbose=True,
**basic_hyperparams
)

In [ ]:

def run_active_sampling_stagnant(params_list, data, sort=False ,verbose=False, **hyperparams):
    results_list = []
    if not verbose:
        warnings.filterwarnings("ignore", category=ConvergenceWarning)
    
    for i, param in enumerate(tqdm(params_list, desc="Running stagnant active sampling", disable=verbose)):
        X, y, scaler, reverse_scaling = prepare_training_data(param, data, sort=sort)
        total_data_count = X.shape[0] 
        
        result = active_sampling_loop_with_stagnant_iterations_stop(X, y, **hyperparams)
        result['params'] = param
        result['total_data_count'] = total_data_count 
        results_list.append(result)
        if verbose:
            print(f"Completed parameter {i+1}/{len(params_list)}: R² = {result['final_r2']:.4f}, Samples = {result['total_samples_used']} of {total_data_count}")
    
    warnings.filterwarnings("default", category=ConvergenceWarning)

    return results_list

stagnant_hyperparams = {
    "max_iterations": 15,
    "uncertainty_threshold": 0.05,
    "higher_uncertainty_threshold": 0.1,
    "r2_improvement_threshold": 1e-4,
    "max_stagnant_iterations": 3
}

results_stagnant = run_active_sampling_stagnant(
params_mostData[:3], data_filtered,
verbose =True,
**stagnant_hyperparams
)

#### Results Visualization & Preliminary Conclusions



##### Visualization 


In this cell, we visualize the aggregated results. The visualization function now accepts:
- `loop_type`: A string indicating the active sampling variant (e.g., "Basic Active Loop" or "Stagnant Active Loop").
- `hyperparams`: A dictionary of hyperparameter values used for the active sampling loop.

The function then creates a title that includes these hyperparameter values and displays three bar charts:
- Final `R²` scores.
- Total samples used.
- Data utilization percentages (samples used as a percentage of total available data).



In [ ]:
import matplotlib.pyplot as plt

def visualize_aggregated_results(results_list, loop_type, hyperparams):
    title_str = f"{loop_type} Results"
    
    hyperparams_items = [f"{k}: {v}" for k, v in hyperparams.items()]
    hyperparams_str = ""
    for i in range(0, len(hyperparams_items), 3):
        chunk = " | ".join(hyperparams_items[i:i+3])
        hyperparams_str += chunk + ("\n" if i + 3 < len(hyperparams_items) else "")
    
    final_r2_list = [r['final_r2'] for r in results_list]
    samples_used_list = [r['total_samples_used'] for r in results_list]
    total_data_counts = [r['total_data_count'] for r in results_list]
    data_utilization_list = [ (used / total) * 100 for used, total in zip(samples_used_list, total_data_counts)]
    
    n = len(results_list)
    indices = range(n)
    
    avg_r2 = sum(final_r2_list) / n
    avg_samples = sum(samples_used_list) / n
    avg_utilization = sum(data_utilization_list) / n
    
    # Create subplots for the aggregated metrics.
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Bar chart for final R².
    axes[0].bar(indices, final_r2_list, color='skyblue')
    axes[0].axhline(avg_r2, color='red', linestyle='--', label=f'Avg R² = {avg_r2:.4f}')
    axes[0].set_xlabel('Parameter Combination Index')
    axes[0].set_ylabel('Final R²')
    axes[0].set_title('Final R² for Each Combination')
    axes[0].legend()
    
    # Bar chart for total samples used.
    axes[1].bar(indices, samples_used_list, color='lightgreen')
    axes[1].axhline(avg_samples, color='red', linestyle='--', label=f'Avg Samples = {avg_samples:.2f}')
    axes[1].set_xlabel('Parameter Combination Index')
    axes[1].set_ylabel('Total Samples Used')
    axes[1].set_title('Samples Used per Combination')
    axes[1].legend()
    
    # Bar chart for data utilization (%).
    axes[2].bar(indices, data_utilization_list, color='plum')
    axes[2].axhline(avg_utilization, color='red', linestyle='--', label=f'Avg Utilization = {avg_utilization:.2f}%')
    axes[2].set_xlabel('Parameter Combination Index')
    axes[2].set_ylabel('Data Utilization (%)')
    axes[2].set_title('Data Utilization per Combination')
    axes[2].legend()
    
    # Set a prominent super-title at the top.
    plt.suptitle(title_str, fontsize=20, fontweight='bold', color='navy', y=0.98)
    
    # Add a centered hyperparameters container below the title.
    # This container is placed at y=0.93 and is horizontally centered (x=0.5).
    plt.gcf().text(0.5, 0.85, f"Hyperparameters:\n\n{hyperparams_str}", 
                   fontsize=12, ha='center', va='center',
                   bbox=dict(facecolor='lightgrey', alpha=0.6, boxstyle='round,pad=0.5'))
    
    # Adjust the layout so that subplots start below the hyperparameters container.
    plt.tight_layout(rect=[0, 0, 1, 0.83])
    plt.show()
    
    print(f"Average Final R²: {avg_r2:.4f}")
    print(f"Average Samples Used: {avg_samples:.2f}")
    print(f"Average Data Utilization: {avg_utilization:.2f}%")

In [ ]:
results_basic = run_active_sampling_basic(params_mostData, data_filtered, verbose=False, **basic_hyperparams)
visualize_aggregated_results(results_basic, "Basic Active Loop", basic_hyperparams)

In [ ]:
results_stagnant = run_active_sampling_stagnant(params_mostData, data_filtered, verbose=False, **stagnant_hyperparams)
visualize_aggregated_results(results_stagnant, "Stagnant Active Loop", stagnant_hyperparams)

##### Preliminary Conclusions

**Results**:

![Basic results updated](./images/step-4-Basic-initial-results.png)

![Stagnant results](./images/step-4-stagnat-initial-results.png)  

**Basic V.S. Stagnant**

- **Basic Active Sampling Loop:**
  - **Robustness:** Tends to use around 18 samples per parameter combination, yielding high `R²` scores (often 1.0000).
  - **Efficiency:** Uses more data, which may be unnecessary in cases with abundant information.

- **Stagnant Active Sampling Loop:**
  - **Efficiency:** Stops early when `R²` improvement stagnates, generally using fewer samples (between 11–16) and still achieving high performance in most cases.
  - **Risks:** In some cases (e.g., parameter combinations with very few total data points), the loop stops after only 3 samples, leading to negative `R²` values and clear underfitting.

 **Outliers and Data Issues**

- **Outlier Cases:**  
  Some parameter combinations consistently use only 3 samples and report poor performance (e.g., negative `R²`). These cases likely represent scenarios where the available data is sparse or noisy, right now its looking like  the most promising direction to investigate and improve.

- **Implication:**  
  Relying solely on the stagnant loop's stopping criterion can result in premature termination. A minimum sample threshold may be necessary to ensure that the model has enough data to learn meaningful patterns.

 **Next Steps**

1. **Revisit the Data:**  
   Analyze the outlier parameter combinations with low total data counts. Determine if these cases should be treated differently or if more data can be acquired.

2. **Implement a Minimum Sample Threshold:**  
   Adjust the stagnant loop to enforce a minimum number of samples (e.g., at least 5 or 7 samples) before allowing early termination (probably most promising result).

3. **Hyperparameter Tuning:**  
   Experiment with different settings for the uncertainty thresholds and maximum stagnant iterations to balance efficiency and robustness. Fine-tuning may reduce instances where the model stops too early.


### Final Optimization and Data Revision

In this section, would try to improve our results and address the points raised in the [preliminary conclusions](#preliminary-conclusions)

- Revisit the Data
- Implement a Minimum Sample Threshold
- Hyperparameter Tuning

#### Revisit the data

##### Revisit Parameter Sampling


Here we re-calculate the parameter combinations using our two strategies:
- **onePerHotel:** For each hotel, we select the combination (of `Snapshot Date`, `Hotel Name`, and `Discount Code`) with the highest transaction count.
- **mostData:** We select the top 40 overall combinations based on transaction counts.

We then display these results in a table (including the transaction counts) so we can review the distribution and identify any potential outliers in data availability.


In [ ]:
grouped = data_filtered.groupby(
    ['Snapshot Date', 'Hotel Name', 'Discount Code']
).size().reset_index(name='transaction_count')

onePerHotel = (
    grouped.sort_values(['Hotel Name', 'transaction_count'], ascending=[True, False])
    .groupby('Hotel Name')
    .first()
    .reset_index()
)
onePerHotel = onePerHotel.sort_values('transaction_count', ascending=False).head(40)

mostData = grouped.sort_values('transaction_count', ascending=False).head(40)

print("One Per Hotel (Top 40) Parameter Combinations with Transaction Counts:")
display(onePerHotel)

print("\nMost Data (Top 40) Parameter Combinations with Transaction Counts:")
display(mostData)


This looks solid to me, shows again the for maximum data we rather stick to Most Data params instead of One Per Hotel

##### Investigating Valid Data and Outliers for Parameter Combinations

Here, we further analyze the data returned by `load_relevant_data` for each parameter combination. For each combination, we compute:
- Total rows (number of transactions)
- Unique counts for WeekDays, Snapshot IDs, Check-in Dates, and Available Rooms
- Weekend count and Holiday-Connected count (using US holidays for 2014–2016, with a check for a holiday occurring within 3 days before or 1 day after the check-in date)
- Data range (from snapshot date to snapshot date + 30 days)
- **Price Metrics:**
  - Minimum and maximum Discount Price
  - Price range as a percentage
  - An array of price points (each price point is `[price, showcount]`)

We then split the parameter combinations into two groups (Good Predictions vs. Suspected Outliers) based on the final `R²` value and display an average comparison table.

In [ ]:
%%capture
%pip install holidays

In [ ]:
import pandas as pd
import holidays

# Create a US holidays object for the years 2014-2016.
us_holidays = holidays.US(years=[2014, 2015, 2016])

def is_connected_to_holiday(checkin_date, window_before=3, window_after=1):
    checkin_date = pd.to_datetime(checkin_date)
    start_window = checkin_date - pd.Timedelta(days=window_before)
    end_window = checkin_date + pd.Timedelta(days=window_after)
    for holiday_date in us_holidays.keys():
        holiday_ts = pd.to_datetime(holiday_date)
        if start_window <= holiday_ts <= end_window:
            return True
    return False

summary_metrics = []
for result in results_stagnant:  # (Use results from the stagnant or basic loop as needed)
    param = result['params']
    df_subset = load_relevant_data(param, data_filtered)
    total_rows = df_subset.shape[0]
    
    unique_weekdays = df_subset['WeekDay'].nunique() if 'WeekDay' in df_subset.columns else None
    unique_snapshot_ids = df_subset['Snapshot ID'].nunique() if 'Snapshot ID' in df_subset.columns else None
    unique_checkin_dates = df_subset['Checkin Date'].nunique() if 'Checkin Date' in df_subset.columns else None
    unique_rooms = df_subset['Available Rooms'].nunique() if 'Available Rooms' in df_subset.columns else None
    
    weekend_count = df_subset[df_subset['WeekDay'].isin(["Friday", "Saturday", "Sunday"])].shape[0]
    holiday_connected_count = df_subset[df_subset['Checkin Date'].apply(is_connected_to_holiday)].shape[0]
    
    snapshot_dt = pd.to_datetime(param['Snapshot Date'])
    data_range = f"{snapshot_dt.strftime('%Y-%m-%d')} - {(snapshot_dt + pd.Timedelta(days=30)).strftime('%Y-%m-%d')}"
    
    # Price metrics
    if total_rows > 0:
        discount_price_min = df_subset["Discount Price"].min()
        discount_price_max = df_subset["Discount Price"].max()
        discount_price_mean = df_subset["Discount Price"].mean()
        discount_price_std = df_subset["Discount Price"].std()
        price_range_percent = ((discount_price_max - discount_price_min) / discount_price_min) * 100 if discount_price_min != 0 else None
        price_counts = df_subset["Discount Price"].value_counts().sort_index()
        price_points = [[price, count] for price, count in price_counts.items()]
    else:
        discount_price_min = discount_price_max = discount_price_mean = discount_price_std = price_range_percent = None
        price_points = []
    
    param_key = f"{param['Snapshot Date']} | {param['Hotel Name']} | {param['Discount Code']}"
    
    summary_metrics.append({
        "Parameter": param_key,
        "Final R²": result['final_r2'],
        "Total Rows": total_rows,
        "Unique WeekDays": unique_weekdays,
        "Unique Snapshot IDs": unique_snapshot_ids,
        "Unique Checkin Dates": unique_checkin_dates,
        "Unique Rooms": unique_rooms,
        "Weekend Count": weekend_count,
        "Holiday Connected Count": holiday_connected_count,
        "Data Range": data_range,
        "Price Points": price_points,
        "Price Points length": len(price_points),
    })

df_metrics = pd.DataFrame(summary_metrics)

# Split into groups based on final R² (good predictions vs. suspected outliers)
df_predicting_well = df_metrics[df_metrics["Final R²"] >= 0]
df_outliers = df_metrics[df_metrics["Final R²"] < 0]

# Compute averages for numeric columns (price metrics will be averaged only for percentage metrics)
avg_good = df_predicting_well.mean(numeric_only=True)
avg_bad = df_outliers.mean(numeric_only=True)

avg_table = pd.DataFrame({"Good Predictions": avg_good, "Bad Predictions": avg_bad})
print("Average Metrics Comparison:")
display(avg_table)

print("\nDetailed Metrics for Good Predictions:")
display(df_predicting_well)
print("\nDetailed Metrics for Suspected Outliers:")
display(df_outliers)


 **Average Metrics Comparison**

| Metric                      | Good Predictions | Bad Predictions  |
|-----------------------------|------------------|------------------|
| Final R²                    | 0.999862         | -0.603804        |
| Total Rows                  | 27.677419        | 24.888889        |
| Unique WeekDays             | 6.419355         | 7.000000         |
| Unique Snapshot IDs         | 1.354839         | 1.111111         |
| Unique Checkin Dates        | 22.580645        | 23.666667        |
| Unique Rooms                | 5.225806         | 9.333333         |
| Weekend Count               | 9.741935         | 11.000000        |
| Holiday Connected Count     | 2.548387         | 4.111111         |
| Price Points Length         | 6.419355         | 2.888889         |

*Note: The current dataset used for model training did not include the "holiday connected" feature. The holiday connected count is computed solely for this analysis.*

**Key Observations**

- **Overall Quality:**  
  - **Similarities:**  
  Both groups have comparable totals for rows, unique weekdays, snapshot IDs, and check-in dates.

  - **Differences:**  
  The outlier group shows notably higher variability in unique rooms (9.33 vs. 5.23) and a higher holiday connected count (4.11 vs. 2.55).

- **Outlier Behavior: Two Subgroups **  
  1. **The Carlyle A Rosewood Group:**  
   - **Performance:** Final R² values are very low or negative.  
   - **Price Data:** Limited price variability (there are only unique 2 price points per data set), with many entries concentrated at one price.  
   - **Other Metrics:** Higher unique rooms and holiday connected counts suggest inconsistencies in the data for this hotel.

  2. **The New York EDITION and Park Hyatt New York Group:**  
    - **Performance:** Despite being flagged as outliers, these cases exhibit high final R² (good model performance).  
    - **Price Data:** They display greater diversity in price points and more consistent room data, which might be influenced by market factors not captured in the current model.



**Next Steps and Conclusions**

- The overall data quality appears acceptable. Although adding the holiday connected feature is an option, my intuition is that it may not significantly improve the model—and could potentially add more noise.
- The limited price variability in the Carlyle A Rosewood subgroup is a promising direction for further investigation.
- The immediate next step should be to examine the algorithm's stopping criteria. In particular, we need to understand why the active sampling stops after only 3 requests and explore the effects of modifying this behavior to request more data points before termination.



#### Why do we stop? (Minimum Sample Threshold)

**Questions**: 
- Why did the model stop sampling after 3 iterations? 
- what would happen if we set a minimum number of iterations (higher)?

##### Debugging the Stop



In this section, we investigate why the basic active sampling loop stops after only a few iterations. We focus on three parameter combinations from the suspected outlier group:
- **params_mostData[9]** (The New York EDITION)
- **params_mostData[21]** (The Carlyle A Rosewood)
- **params_mostData[29]** (Park Hyatt New York)

We use a debug version of the active sampling loop that prints:
- The training and candidate indices at each iteration.
- The uncertainty values for each candidate.
- The final training Data.
- The maximum uncertainty and the chosen candidate index.
- The current `R²` score.

This information should shed light on why the loop is stopping early, helping us determine if a higher minimum sample threshold might be necessary.


In [ ]:
def active_sampling_loop_debug_table(X, y, max_iterations=10, uncertainty_threshold=0.05, initial_sample_indices=None):
    n_samples = X.shape[0]
    if initial_sample_indices is None:
        initial_sample_indices = [0, n_samples // 2, n_samples - 1]
    
    training_indices = set(initial_sample_indices)
    candidate_indices = set(range(n_samples)) - training_indices
    iteration_log = []
    
    for iteration in range(max_iterations):
        X_train = X.iloc[sorted(list(training_indices))]
        y_train = y.iloc[sorted(list(training_indices))]

        model = initialize_model()
        model.fit(X_train, y_train)
        
        candidate_list = sorted(list(candidate_indices))
        X_candidates = X.iloc[candidate_list]
        y_pred_candidates, std_candidates = model.predict(X_candidates, return_std=True)
        
        max_std = np.max(std_candidates)
        chosen_candidate_idx = candidate_list[np.argmax(std_candidates)]
        
        y_pred_full = model.predict(X)
        current_r2 = r2_score(y, y_pred_full)
        
        iteration_details = {
            "Iteration": iteration,
            "Training Indices": ", ".join(map(str, sorted(training_indices))),
            "Candidate Indices": ", ".join(map(str, sorted(candidate_indices))),
            "Candidate Uncertainties": str(np.round(std_candidates, 4).tolist()),
            "Max Uncertainty": round(max_std, 4),
            "Chosen Candidate": chosen_candidate_idx,
            "Current R²": round(current_r2, 4)
        }
        iteration_log.append(iteration_details)
        
        if max_std < uncertainty_threshold:
            break
        
        training_indices.add(chosen_candidate_idx)
        candidate_indices.remove(chosen_candidate_idx)
    
    X_final = X.iloc[sorted(list(training_indices))]
    y_final = y.iloc[sorted(list(training_indices))]
    final_model = initialize_model()
    final_model.fit(X_final, y_final)
    final_predictions = final_model.predict(X)
    final_r2 = r2_score(y, final_predictions)
    
    final_details = {
        "Final R²": round(final_r2, 4),
        "Total Samples Used": len(training_indices)
    }
    
    print("Iteration Details:")
    display(pd.DataFrame(iteration_log))
    print("\nFinal Details:")
    display(pd.DataFrame([final_details]))
    
    print("\nFinal Training Data (first 10 rows):")
    display(X_final.head(10))
    print("Final Training Target (first 10 rows):")
    display(y_final.head(10))
    
    return {
        'final_model': final_model,
        'final_predictions': final_predictions,
        'final_r2': final_r2,
        'iteration_log': iteration_log,
        'total_samples_used': len(training_indices)
    }

# Debug the basic loop for selected parameters from the suspected outlier group.
params_to_debug = [params_mostData[9], params_mostData[21], params_mostData[29]]

for param in params_to_debug:
    print("--------------------------------------------------")
    print(f"Debugging for parameter: {param['Snapshot Date']} | {param['Hotel Name']} | {param['Discount Code']}")
    X_debug, y_debug, scaler, reverse_scaling = prepare_training_data(param, data_filtered)
    result_debug = active_sampling_loop_debug_table(X_debug, y_debug, max_iterations=15, uncertainty_threshold=0.05)
    print("--------------------------------------------------\n")


**Analysis and Improvements: Understanding R² and Model Behavior**

 **What We Found**
- **Initial Sample Homogeneity:**  
  We observed that the three initial samples (selected using indices `[0, n_samples // 2, n_samples - 1]`) in out outliers cased often turn out to be very similar. having the same discount price. 
  - This causes the model to quickly learn a prediction near initial indices discount price.

 **A Reminder About R²**
- **R² (Coefficient of Determination):**  
  Measures how well the model explains the variance in the target variable.
  - **R² ≈ 1:** The model explains nearly all the variability.
  - **R² ≈ 0:** The model performs no better than simply predicting the mean.
  - **Negative R²:** The model performs worse than predicting the mean.

 **Illustrative Examples**
1. **Example 1:** `[1, 3, 5, 7, 1, 8, 4, 7, 1]`
   - **Average Calculation:**  
     Sum = 37, Average ≈ 4.11  
   - **Scenario:**  
     If the model starts with three initial samples that are all `1`, it learns that the target is about 1.  
     However, because the overall average is 4.11, predicting 1 is far off, resulting in a very poor model fit (negative R²).

2. **Example 2:** `[1, 1, 1, 1, 1, 1, 1, 3, 1]`
   - **Average Calculation:**  
     Sum = 11, Average ≈ 1.22  
   - **Scenario:**  
     If the model again starts with three `1`s, it learns that the target is about 1.  
     Here, because the overall average is close to 1 (1.22), the prediction is nearly as good as the average, leading to an R² close to 0.

These examples illustrate that when initial samples are too similar, the model essentially learns to predict a single value—its prediction is then almost equivalent to the mean, causing R² to be very low or even negative.

 **How to Improve the Model**
**Potential Strategies:**
- **Quantile-Based (Sorted) Sampling:**  
  - *Pros:* Ensures that the initial samples cover the full range of discount prices.
  - *Cons:* Requires additional preprocessing to determine quantile indices. im most cases this would lead to a trade of with sampling amount.
- **Increasing the Initial Sample Count:**  
  - *Pros:* Captures more variability by starting with more diverse data points.
  - *Cons:* Increases computational cost in early iterations + may not solve the problem in all cases (see example 2).
- **Random or Stratified Sampling:**  
  - *Pros:* Reduces bias and ensures different segments of the data are represented.
  - *Cons:* May yield inconsistent results unless a fixed random seed is used + may not solve the problem in all cases (see example 2).

**Current Approach:**  
We currently sort the sub-dataset by discount price so that the initial samples are drawn from different parts of the distribution. However, when the data are naturally homogeneous, this method may still select very similar points.

 **Summary**
When the initial sample points are too similar, the model essentially learns to predict a value near the average, leading to an R² near 0 or even negative scores. In contrast, a more diverse set of initial samples could improve the model’s ability to capture the data’s variability. Our next steps will focus on refining the sampling strategy—possibly using quantile-based or stratified sampling—and exploring a higher minimum sample threshold to achieve better model performance.


##### 

### Sorting the data and visualizing the results

I add the sort option to `prepare_training_data`, if sort=true the data will be sorted by discount price.
```py

if sorted:
        normalized = normalized.sort_values(by="Discount Price")

```

and added the same option to `run_active_sampling_stagnant` and `run_active_sampling_basic`.

lets see the result.

In [ ]:
results_basic = run_active_sampling_basic(params_mostData, data_filtered, sort=True, verbose=False, **basic_hyperparams)
visualize_aggregated_results(results_basic, "Basic - sorted", basic_hyperparams)

In [ ]:
results_stagnant = run_active_sampling_stagnant(params_mostData, data_filtered, sort=True ,verbose=False, **stagnant_hyperparams)
visualize_aggregated_results(results_stagnant, "Stagnant - sorted", stagnant_hyperparams)

![Basic results](./images/step4-basic-sorted-Result(pre-tuning).png)
![stagnat results](./images/step4-stagnat-sorted-Result(pre-tuning).png)

**Wow!** This improvement is insane! but the data utilization is not so great, 67% percent for basic and 48% for stagnant, we can do better!
Lets fine tune the hyper parameters to lose some performance for less trining.

### Fine-tuning hyperparameters  to get the best R^2 to Data utilization

Our goal is to strike the right balance between model performance (as measured by R²) and data efficiency (minimizing the percentage of available data used).
In our active sampling loops, the stopping criteria play a key role in this trade-off.

 **What We’re Tuning**

**For the Basic Active Sampling Loop:**
- **max_iterations:** The maximum number of iterations allowed.
- **uncertainty_threshold:** The threshold below which the model’s uncertainty is considered low enough to stop sampling.

**For the Stagnant Active Sampling Loop:**
- **max_iterations:** Same as above.
- **uncertainty_threshold:** Primary stopping threshold.
- **higher_uncertainty_threshold:** A secondary threshold used when combined with stagnation.
- **r2_improvement_threshold:** The minimum improvement in R² required between iterations.
- **max_stagnant_iterations:** The maximum number of consecutive iterations with minimal R² improvement before stopping.

**How would we do that?**

We would use **BayesianOptimization** to fine-tune our hyperparameters because it efficiently explores the hyperparameter space by balancing exploration and exploitation. This method requires fewer iterations compared to exhaustive grid search and quickly converges to the best set of parameters that maximize our model's R².

Our objective function returns the final R², and we define bounds for key hyperparameters (like uncertainty thresholds, R² improvement thresholds, and the maximum stagnant iterations). The optimized parameters help improve model performance and data utilization.

#### Fine-Tuning with Bayesian Optimization


**Fine-Tuning Hyperparameters with Bayesian Optimization**

We use Bayesian Optimization to search for the best hyperparameters for our stagnant active sampling loop. Our objective function, `objective_hyperparams_stagnant/basic`, randomly selects 5 parameter combinations from `params_mostData` (with sorted data) and runs the stagnant active sampling loop on each. It then computes the average final R² and average data utilization. A penalty is applied if the average R² falls below 0.96, ensuring that only hyperparameters yielding strong performance are favored.

The optimizer then finds the hyperparameter values that maximize our objective. We save the optimized hyperparameters in a variable (`stagnant/basic_optimized_hyperparameters`) and also pickle them to disk for future use. Finally, we run the stagnant active sampling loop with these optimized parameters and visualize the results.

This process ensures we balance high R² performance with low data utilization.


In [ ]:
%%capture
%pip install bayesian-optimization

##### Basic hyperparameters optimization

In [ ]:
import random
from bayes_opt import BayesianOptimization
import pickle

def objective_hyperparams_basic(uncertainty_threshold, max_iterations):
    max_iterations = int(max_iterations)
    hyperparams = {"max_iterations": max_iterations, "uncertainty_threshold": uncertainty_threshold}

    selected_params = random.sample(params_mostData, 3)
    results_list = run_active_sampling_basic(selected_params, data_filtered, sort=True, verbose=False, **hyperparams)
    
    avg_r2 = sum(result['final_r2'] for result in results_list) / len(results_list)
    utilization_vals = [result['total_samples_used'] / result['total_data_count'] for result in results_list]
    avg_utilization = sum(utilization_vals) / len(utilization_vals)
    score = 1 - avg_utilization
    if avg_r2 < 0.96:
        penalty = (0.96 - avg_r2) * 100
        score -= penalty
    return score

pbounds_basic = {"uncertainty_threshold": (0.01, 0.1), "max_iterations": (10, 30)}

optimizer_basic = BayesianOptimization(f=objective_hyperparams_basic, pbounds=pbounds_basic, random_state=42)
optimizer_basic.maximize(init_points=5, n_iter=150)

basic_optimized_hyperparameters = {k: int(v) if k == "max_iterations" else v for k, v in optimizer_basic.max['params'].items()}

with open("basic_optimized_hyperparameters.pkl", "wb") as f:
    pickle.dump(basic_optimized_hyperparameters, f)


In [ ]:
with open("basic_optimized_hyperparameters.pkl", "rb") as f:
    basic_optimized_hyperparameters = pickle.load(f)

results_basic = run_active_sampling_basic(params_mostData, data_filtered, sort=True, verbose=False, **basic_optimized_hyperparameters)
visualize_aggregated_results(results_basic, "Basic - sorted - optimized ", basic_optimized_hyperparameters)

##### Stagnant hyperparameters optimization

In [ ]:
import random
from bayes_opt import BayesianOptimization
import pickle

def objective_hyperparams_stagnant(uncertainty_threshold, higher_uncertainty_threshold, r2_improvement_threshold, max_stagnant_iterations, max_iterations):
    max_stagnant_iterations = int(max_stagnant_iterations)
    max_iterations = int(max_iterations)
    hyperparams = {
        "max_iterations": max_iterations,
        "uncertainty_threshold": uncertainty_threshold,
        "higher_uncertainty_threshold": higher_uncertainty_threshold,
        "r2_improvement_threshold": r2_improvement_threshold,
        "max_stagnant_iterations": max_stagnant_iterations,
    }
    selected_params = random.sample(params_mostData, 3)
    results_list = run_active_sampling_stagnant(selected_params, data_filtered, sort=True, verbose=False, **hyperparams)
    avg_r2 = sum(result['final_r2'] for result in results_list) / len(results_list)
    utilization_vals = [result['total_samples_used'] / result['total_data_count'] for result in results_list]
    avg_utilization = sum(utilization_vals) / len(utilization_vals)
    score = 1 - avg_utilization
    if avg_r2 < 0.96:
        penalty = (0.96 - avg_r2) * 100
        score -= penalty
    return score

pbounds = {
    "uncertainty_threshold": (0.01, 0.1),
    "higher_uncertainty_threshold": (0.05, 0.2),
    "r2_improvement_threshold": (1e-5, 1e-3),
    "max_stagnant_iterations": (2, 5),
    "max_iterations": (10, 30),
}

optimizer = BayesianOptimization(
    f=objective_hyperparams_stagnant,
    pbounds=pbounds,
    random_state=42,
)
optimizer.maximize(init_points=5, n_iter=150)

stagnant_optimized_hyperparameters = {
    k: int(v) if k in ['max_iterations', 'max_stagnant_iterations'] else v 
    for k, v in optimizer.max['params'].items()
}

with open("stagnant_optimized_hyperparameters.pkl", "wb") as f:
    pickle.dump(stagnant_optimized_hyperparameters, f)


In [ ]:
import pickle

with open("stagnant_optimized_hyperparameters.pkl", "rb") as f:
    stagnant_optimized_hyperparameters = pickle.load(f)

results_stagnant = run_active_sampling_stagnant(params_mostData, data_filtered, sort=True ,verbose=False, **stagnant_optimized_hyperparameters)
visualize_aggregated_results(results_stagnant, "Stagnant - sorted - optimized", stagnant_optimized_hyperparameters)


##### Hyperparameters Fine-Tuning Result Analysis

After fine-tuning our hyperparameters using Bayesian Optimization, we achieved impressive improvements in data utilization without sacrificing model performance.

**Results visualized**

**Before optimization:**
![Basic results pre](./images/step4-basic-sorted-Result(pre-tuning).png)
![stagnat results pre](./images/step4-stagnat-sorted-Result(pre-tuning).png)


**After optimization:**
![Basic results post](./images/step4-optimized-basic.png)
![stagnat results post](./images/step4-optimized-stagnat.png)


**Compression**

| **Model**    | **Metric**           | **Before Tuning**               | **After Tuning**                | **Percentage Change**        |
|--------------|----------------------|---------------------------------|---------------------------------|------------------------------|
| **Stagnant** | Data Utilization     | 48.24% (~12.78 samples/run)     | 37.52% (~9.95 samples/run)         | ~25% reduction               |
|              | R² Reduction         | 0% drop                         | ~0.13% drop (<0.0015 reduction)   | ~0.13% drop (negligible)     |
| **Basic**    | Data Utilization     | 67.82% (~18 samples/run)      | 42.94% (~11.35 samples/run)         | ~36% reduction               |
|              | R² Reduction         | 0% drop                         | No visible drop                | 0% change                   |


**Summary**

We are very pleased with the final results. With the tuned hyperparameters, our stagnant model now uses only about 10 samples per run to predict the entire month’s prices with nearly 100% accuracy. This level of efficiency and performance is truly remarkable and demonstrates the strength of our approach in balancing model accuracy with data efficiency.

## step 5 - PySpark & Mllib for step 3

### Installation

Spark requires a java version of 8 or 11 to work

In [ ]:
%%capture 
!conda install -c conda-forge openjdk=11 -y

config spark to work with our env java version(11)

In [ ]:
import os 
java_home_path = os.popen('dirname $(dirname $(which java))').read().strip()
os.environ["JAVA_HOME"] = java_home_path
print(f"JAVA_HOME is set to: {os.environ['JAVA_HOME']}")


In [ ]:
%%capture 
%pip install pyspark plotly

### Create Spark session

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, lit, concat, first, array, udf
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import BisectingKMeans
from pyspark.sql.functions import min as spark_min

try:
    spark.stop()
except:
    pass

spark = SparkSession.builder.appName("HotelClustering").getOrCreate()
print("Create Spark session") 

### Data Loading and Filtering

- Load the CSV file.
- Select the top 150 hotels (by number of records).
- Select the top 40 checkin dates.

In [ ]:
file_path = "./hotels_data_changed.csv"

df = spark.read.option("header", "true").option("inferSchema", "true").csv(file_path)

top150_hotels = df.groupBy("Hotel Name").agg(count("*").alias("cnt")) .orderBy(col("cnt").desc()).limit(150)


df_top150 = df.join(top150_hotels.select("Hotel Name"), on="Hotel Name", how="inner")


top40_dates = df_top150.groupBy("Checkin Date").agg(count("*").alias("cnt")).orderBy(col("cnt").desc()).limit(40)

df_top150_dates = df_top150.join(top40_dates.select("Checkin Date"), on="Checkin Date", how="inner")

df_top150_dates.head()


### Convert to 160-dims vector

In [ ]:
df_grouped = df_top150_dates.groupBy("Hotel Name", "Checkin Date", "Discount Code").agg(spark_min("Discount Price").alias("minDiscountPrice"))

df_grouped = df_grouped.withColumn("date_code", concat(col("Checkin Date"), lit("_"), col("Discount Code")))

df_pivot = df_grouped.groupBy("Hotel Name").pivot("date_code").agg(first("minDiscountPrice"))

df_pivot = df_pivot.fillna(-1)

df_pivot.head(3)

We might find ourself in a situation where some of the hotels don't have a column for all the 160 dates + discount codes.
So we would ensure thy all have that.

In [ ]:
top40_list = [row["Checkin Date"] for row in top40_dates.collect()]
discount_codes = [1, 2, 3, 4]

# Build the expected column names (format: "YYYY-MM-DD_1", etc.)
expected_cols = [f"{date}_{code}" for date in top40_list for code in discount_codes]

# Add any missing expected columns with default -1
existing_cols = df_pivot.columns
for col_name in expected_cols:
    if col_name not in existing_cols:
        df_pivot = df_pivot.withColumn(col_name, lit(-1))

# Reorder the DataFrame columns so that they appear in the desired order:
df_pivot = df_pivot.select(["Hotel Name"] + expected_cols)
df_pivot.head(3)

### Normalization and Save to CSV

Normalize the 160 price columns row-by-row (scaling valid prices to a 0–100 range, leaving missing values as -1).

In [ ]:
# Combine the 160 price columns into an array column
df_pivot = df_pivot.withColumn("prices_array", array(*expected_cols))

# Define a UDF for normalizing the prices for each hotel (ignoring -1 values)
def normalize_prices(prices):
    # Filter out missing values (-1)
    valid_prices = [p for p in prices if p != -1]
    if not valid_prices:
        return prices
    min_price = min(valid_prices)
    max_price = max(valid_prices)
    if min_price == max_price:
        return [0 if p != -1 else -1 for p in prices]
    normalized = []
    for p in prices:
        if p == -1:
            normalized.append(-1)
        else:
            norm_val = round(((p - min_price) / (max_price - min_price)) * 100)
            normalized.append(int(norm_val))
    return normalized

normalize_udf = udf(normalize_prices, ArrayType(IntegerType()))

# Apply the normalization UDF to create a new column with normalized prices.
df_pivot = df_pivot.withColumn("norm_prices_array", normalize_udf("prices_array"))

# Replace the original price columns with the normalized values.
for i, col_name in enumerate(expected_cols):
    df_pivot = df_pivot.withColumn(col_name, col("norm_prices_array")[i])

# Optionally, drop helper columns.
df_final = df_pivot.drop("prices_array", "norm_prices_array")


**Save to CSV**

In [ ]:
pyspark_hotels_clustering_data= "./pyspark_hotels_clustering_data.csv"
df_final.write.option("header", "true").mode("overwrite").csv(pyspark_hotels_clustering_data)


### Clustering

#### Read CSV and assemble features

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Define the path to the CSV folder that was saved in Step 5.
pyspark_hotels_clustering_data = "./pyspark_hotels_clustering_data.csv"

# Read the CSV data (Spark will read all part files in the folder)
df_loaded = spark.read.option("header", "true").option("inferSchema", "true").csv(pyspark_hotels_clustering_data)

# Identify the feature columns (all columns except "Hotel Name")
feature_cols = [col for col in df_loaded.columns if col != "Hotel Name"]

# Assemble the 160 normalized price columns into a single feature vector.

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_features = assembler.transform(df_loaded)


#### Clustering with MLlib (BisectingKMeans)

In [ ]:
from pyspark.ml.clustering import BisectingKMeans

# Set the number of clusters (adjust k as needed)
bkmeans = BisectingKMeans(featuresCol="features", predictionCol="cluster", k=4)

# Train the model
model = bkmeans.fit(df_features)

# Add the cluster assignments to the DataFrame
df_clustered = model.transform(df_features)

# Show the hotel names along with their cluster assignments
df_clustered.select("Hotel Name", "cluster").show(truncate=False)


#### Visualization with PCA and Plotly

Since the features are 160-dimensional, we use PCA to reduce them to 2 dimensions for visualization. Then, we convert the Spark DataFrame to a Pandas DataFrame and use Plotly Express to create a scatter plot.

In [ ]:
from pyspark.ml.feature import PCA
pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")
pca_model = pca.fit(df_clustered)
df_pca = pca_model.transform(df_clustered)

pandas_df = df_pca.select("Hotel Name", "cluster", "pcaFeatures").toPandas()

# Split the PCA features into two separate columns for plotting
pandas_df["pca1"] = pandas_df["pcaFeatures"].apply(lambda x: x[0])
pandas_df["pca2"] = pandas_df["pcaFeatures"].apply(lambda x: x[1])


In [ ]:
import plotly.express as px

fig = px.scatter(
    pandas_df,
    x="pca1",
    y="pca2",
    color="cluster",
    hover_data=["Hotel Name"],
    title="Hotel Clusters Visualization (PCA Reduced)"
)
fig.show()


#### Stop the spark session

In [ ]:
spark.stop()